# Preliminaries and checkers 

In [5]:
!pip install gymnasium
!pip install torch
!pip install "ray[rllib]==2.38.0"
!pip install "ray[rllib,torch]==2.38.0"
!pip install "ray[all]==2.38.0"
!pip install pygame
!pip install matplotlib
!pip install networkx

  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0


In [ ]:
!pip uninstall "ray[rllib]==2.38.0"
!pip uninstall "ray[rllib,torch]==2.38.0"
!pip uninstall "ray[all]==2.38.0"

In [1]:
import ray
ray.init(ignore_reinit_error=True,num_gpus=1)
print(ray.cluster_resources())

2024-12-02 15:52:01,005	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-12-02 15:52:08,904	INFO worker.py:1807 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


{'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'CPU': 12.0, 'object_store_memory': 3984535142.0, 'node:127.0.0.1': 1.0, 'memory': 7969070286.0, 'GPU': 1.0}


In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.version.cuda)


True
0
12.4


# Relevant and working codes 

In [ ]:
import numpy as np
import networkx as nx
import gymnasium as gym
from gymnasium import spaces
import ray
from ray import tune
from ray.rllib.algorithms.ppo import PPO
from ray.rllib.utils.framework import try_import_torch
from ray.rllib.env.multi_agent_env import MultiAgentEnv
import sys
import torch
import os
from collections import deque
import time
import pygame
import xml.etree.ElementTree as ET
import re

# Set the CUDA_VISIBLE_DEVICES environment variable to the GPU index of the L4 (e.g., 0)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Replace "0" with the actual index of the L4 GPU based on `nvidia-smi`

pygame.init()
WIDTH, HEIGHT = 1200, 900
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Aircraft Movement on Taxiways")

# Parse the KML file
file_path = "5x5.kml"  # Update this path to your KML file
tree = ET.parse(file_path)
root = tree.getroot()
namespace = {'kml': 'http://www.opengis.net/kml/2.2'}

# Lists to store the taxiways and gates
taxiways = []
gates = []

# Extract LineStrings (taxiways and runways) and Points (gates)
for placemark in root.findall('.//kml:Placemark', namespace):
    line_string = placemark.find('.//kml:LineString/kml:coordinates', namespace)
    if line_string is not None:
        line_coords = line_string.text.strip().split()
        path = []
        for coord in line_coords:
            lon, lat, _ = map(float, coord.split(','))
            path.append((lon, lat))
        taxiway = {'path': path}
        name = placemark.find('kml:name', namespace)
        if name is not None:
            placemark_name = name.text.strip()
            taxiway['name'] = placemark_name
            match = re.search(r'\b(L1|L2|L3)\b', placemark_name)
            if match:
                taxiway['code'] = match.group(1)
            else:
                taxiway['code'] = ''
            if 'CENTRELINE' in placemark_name.upper():
                taxiway['is_runway'] = True
        taxiways.append(taxiway)
    point = placemark.find('.//kml:Point/kml:coordinates', namespace)
    if point is not None:
        coord = point.text.strip().split(',')
        lon, lat, _ = map(float, coord)
        gates.append({'position': (lon, lat)})

# Find the bounds of all coordinates
all_taxiway_coords = [coord for taxiway in taxiways for coord in taxiway['path']]
all_gate_coords = [gate['position'] for gate in gates]
all_coords = all_taxiway_coords + all_gate_coords

lon_min = min(coord[0] for coord in all_coords)
lon_max = max(coord[0] for coord in all_coords)
lat_min = min(coord[1] for coord in all_coords)
lat_max = max(coord[1] for coord in all_coords)

margin = 0.05
lon_range = lon_max - lon_min
lat_range = lat_max - lat_min
lon_min -= lon_range * margin
lon_max += lon_range * margin
lat_min -= lat_range * margin
lat_max += lat_range * margin

def convert_coords(lon, lat):
    x = (lon - lon_min) / (lon_max - lon_min) * WIDTH
    y = HEIGHT - (lat - lat_min) / (lat_max - lat_min) * HEIGHT
    return pygame.math.Vector2(x, y)

for taxiway in taxiways:
    pixel_path = [convert_coords(lon, lat) for lon, lat in taxiway['path']]
    taxiway['pixel_path'] = pixel_path

for gate in gates:
    gate['pixel_position'] = convert_coords(*gate['position'])

def distance(p1, p2):
    return (p1 - p2).length()

G = nx.Graph()
node_id_counter = 0
node_positions = {}
for idx, taxiway in enumerate(taxiways):
    path = taxiway['pixel_path']
    taxiway_name = taxiway.get('name', '')
    taxiway_code = taxiway.get('code', '')
    taxiway_nodes = []
    for i, point in enumerate(path):
        node_id = node_id_counter
        node_id_counter += 1
        G.add_node(node_id, pos=(point.x, point.y))
        G.nodes[node_id]['taxiway_name'] = taxiway_name
        G.nodes[node_id]['taxiway_code'] = taxiway_code
        if taxiway.get('is_runway'):
            G.nodes[node_id]['is_runway_node'] = True
        node_positions[node_id] = point
        taxiway_nodes.append(node_id)
        if i > 0:
            prev_node_id = taxiway_nodes[i - 1]
            G.add_edge(prev_node_id, node_id, weight=distance(point, node_positions[prev_node_id]))
    taxiway['node_ids'] = taxiway_nodes

def connect_taxiway_endpoints():
    CONNECTION_THRESHOLD = 30
    for taxiway1 in taxiways:
        for taxiway2 in taxiways:
            if taxiway1 == taxiway2:
                continue
            endpoints1 = [taxiway1['node_ids'][0], taxiway1['node_ids'][-1]]
            endpoints2 = [taxiway2['node_ids'][0], taxiway2['node_ids'][-1]]
            for node1 in endpoints1:
                for node2 in endpoints2:
                    pos1 = pygame.math.Vector2(G.nodes[node1]['pos'])
                    pos2 = pygame.math.Vector2(G.nodes[node2]['pos'])
                    if distance(pos1, pos2) < CONNECTION_THRESHOLD:
                        if not G.has_edge(node1, node2):
                            G.add_edge(node1, node2, weight=distance(pos1, pos2))

connect_taxiway_endpoints()

def connect_runways_to_taxiways():
    CONNECTION_THRESHOLD = 30
    for runway in [t for t in taxiways if t.get('is_runway')]:
        for taxiway in [t for t in taxiways if not t.get('is_runway')]:
            for runway_node in runway['node_ids']:
                for taxiway_node in taxiway['node_ids']:
                    pos_runway = pygame.math.Vector2(G.nodes[runway_node]['pos'])
                    pos_taxiway = pygame.math.Vector2(G.nodes[taxiway_node]['pos'])
                    if distance(pos_runway, pos_taxiway) < CONNECTION_THRESHOLD:
                        if not G.has_edge(runway_node, taxiway_node):
                            G.add_edge(runway_node, taxiway_node, weight=distance(pos_runway, pos_taxiway))

connect_runways_to_taxiways()

flipped_positions = {node: (pos[0], -pos[1]) for node, pos in node_positions.items()}

# Ensure all edges have a weight of 1
for edge in G.edges():
    G[edge[0]][edge[1]]['weight'] = 1

# Custom Multi-Agent Environment for Shortest Path Finding (RLlib-compatible)
class RoadIntersectionEnv(MultiAgentEnv):
    def __init__(self, graph, num_agents=3):
        super().__init__()
        self.graph = graph
        self.total_agents = num_agents
        self.agents = [f"car_{i}" for i in range(self.total_agents)]
        self.pos = {agent: None for agent in self.agents}
        self.distance_travelled = {agent: 0 for agent in self.agents}

        # Use Discrete spaces to represent each agent's current node in the graph
        self.observation_space = spaces.Discrete(len(self.graph.nodes))
        self.action_space = spaces.Discrete(len(self.graph.nodes))

    def reset(self, seed=None, options=None):
        if seed is not None:
            np.random.seed(seed)

        nodes = np.array(list(self.graph.nodes)).flatten()
        for agent in self.agents:
            self.pos[agent] = np.random.choice(nodes)
            self.distance_travelled[agent] = 0
        self.done_agents = set()
        obs = {agent: self.pos[agent] for agent in self.agents}  # Return observations as a dictionary
        return obs, {}

    def step(self, action_dict):
        rewards = {agent: 0 for agent in self.agents}
        dones = {agent: False for agent in self.agents}
        infos = {}
        truncateds = {agent: False for agent in self.agents}

        # Move each agent
        for agent, action in action_dict.items():
            if agent in self.done_agents:
                continue

            current_node = self.pos[agent]
            next_node = action

            # Ensure next_node is a valid neighbor of the current node
            if current_node in self.graph and next_node in self.graph[current_node]:
                self.pos[agent] = next_node
                self.distance_travelled[agent] += nx.get_edge_attributes(self.graph, 'weight').get((current_node, next_node), 1)
                rewards[agent] = -0.1  # Small negative reward to encourage shorter paths  # Encourage shorter paths

        # Detect collisions
        occupied_nodes = {}
        for agent in self.agents:
            if agent in self.done_agents:
                continue

            pos = self.pos[agent]
            if pos not in occupied_nodes:
                occupied_nodes[pos] = agent
            else:
                # Collision occurred
                rewards[agent] -= 5  # Reduced penalty for collision
                other_agent = occupied_nodes[pos]
                rewards[other_agent] -= 10
                dones[agent] = True
                dones[other_agent] = True
                self.done_agents.add(agent)
                self.done_agents.add(other_agent)

        # Update done flags for all agents
        for agent in self.agents:
            if agent not in self.done_agents:
                dones[agent] = False
            else:
                dones[agent] = True
                truncateds[agent] = True  # Mark as truncated to properly handle episode end

        # Check if all agents are done
        all_done = len(self.done_agents) == self.total_agents
        dones['__all__'] = all_done
        truncateds['__all__'] = all_done

        obs = {agent: self.pos[agent] for agent in self.agents if agent not in self.done_agents}  # Return observations as a dictionary
        infos = {agent: {} for agent in self.agents if agent not in self.done_agents}  # Update infos to match the current observation keys

        return obs, rewards, dones, truncateds, infos

def render_pygame(env):
    screen.fill((255, 255, 255))  # Fill background with white

    # Draw taxiways
    for taxiway in taxiways:
        pixel_path = taxiway['pixel_path']
        if len(pixel_path) > 1:
            pygame.draw.lines(screen, (0, 0, 0), False, [(point.x, point.y) for point in pixel_path], 2)

    # Draw nodes
    for node in env.graph.nodes:
        pos = env.graph.nodes[node]['pos']
        pygame.draw.circle(screen, (0, 0, 255), (int(pos[0]), int(pos[1])), 5)

    # Draw agents
    colors = [(255, 0, 0), (0, 255, 0), (128, 0, 128)]
    for idx, agent in enumerate(env.agents):
        if env.pos[agent] is not None:
            pos = env.graph.nodes[env.pos[agent]]['pos']
            pygame.draw.circle(screen, colors[idx % len(colors)], (int(pos[0]), int(pos[1])), 10)

    pygame.display.flip()  # Update the display

# Adjust training config and PPO code accordingly

# Create environment
env = RoadIntersectionEnv(G, num_agents=3)

# RLlib Training Configuration with fixed typo
ray.init(ignore_reinit_error=True, num_gpus=1, log_to_driver=True)

from ray.tune.registry import register_env

def env_creator(env_config):
    return RoadIntersectionEnv(env_config['graph'], env_config['num_agents'])

register_env("RoadIntersectionEnv", lambda env_config: env_creator(env_config))

def policy_mapping_fn(agent_id, *args, **kwargs):
    return "shared_policy"

config = {
    "rllib_version": "2.38",
    "env": "RoadIntersectionEnv",
    "env_config": {
        "graph": G,
        "num_agents": 3
    },
    "framework": "torch",  # Switch to PyTorch framework
    "num_workers": 5,  # Adjust number of workers to utilize available CPU cores
    "num_envs_per_worker": 80,  # Set environments per worker to balance CPU usage
    "num_gpus": 1,  # Specify that GPU should be used
    "train_batch_size": 8000,  # Increased batch size for more data per update
    "sgd_minibatch_size": 128,  # Increase minibatch size for training
    "lr": 1e-4,  # Reduced learning rate for stability
    "gamma": 0.99,  # Discount factor for reward
    "log_level": "DEBUG",
    "multiagent": {
        "policies": {
            "shared_policy": (None, env.observation_space, env.action_space, {})
        },
        "policy_mapping_fn": policy_mapping_fn,
        "policies_to_train": ["shared_policy"],
    },
    "exploration_config": {  # Add exploration configuration
        "type": "EpsilonGreedy",
        "initial_epsilon": 1.0,
        "final_epsilon": 0.05,
        "epsilon_timesteps": 20000  # Increased exploration time for better learning
    },
    "monitor": True,  # Enable monitoring for visualization
    "metric": "episode_reward_mean",
    "mode": "max"
}

# Track training progress
rewards_history = deque(maxlen=100)
iterations = []
mean_rewards = []

# Train the model using PPO with more frequent checkpoints
analysis = tune.run(
    "PPO", 
    config=config,
    reuse_actors=True, 
    stop={"training_iteration": 25}, 
    verbose=1, 
    storage_path="D:/aniru-windows/Capstone-Implementation/Localrun_ray_checkpoints", 
    checkpoint_at_end=True,
    checkpoint_freq=5  # Save checkpoints more frequently
)

# Save the model
try:
    print("Available metrics:", analysis.results_df.columns)
    if analysis.trials and analysis.get_best_trial(metric="training_iteration", mode="max") is not None:
        best_trial = analysis.get_best_trial(metric="training_iteration", mode="max")
        best_checkpoint = analysis.get_best_checkpoint(trial=best_trial, metric="training_iteration", mode="max")
    else:
        # Fallback if 'episode_reward_mean' is not available
        best_trial = analysis.get_best_trial(metric="training_iteration", mode="max")
        best_checkpoint = analysis.get_best_checkpoint(trial=best_trial, metric="training_iteration", mode="max")

    if best_checkpoint is not None:
        print(f"Best checkpoint: {best_checkpoint}")

        # Use the trained model
        obs = env.reset()[0]
        trainer = PPO(env="RoadIntersectionEnv", config=config)
        trainer.restore(best_checkpoint)

        done = {agent: False for agent in env.agents}
        done['__all__'] = False

        should_render = True  # Change to True to visualize

        while not done['__all__']:
            actions = {agent: trainer.compute_single_action(obs[agent], policy_id="shared_policy", explore=True) for agent in env.agents if not done[agent]}
            obs, rewards, done, truncateds, infos = env.step(actions)
            
            if should_render:
                render_pygame(env)

            # Track rewards for visualization
            episode_reward = sum(rewards.values())
            rewards_history.append(episode_reward)
            iterations.append(len(iterations) + 1)
            mean_reward = np.mean(rewards_history)
            mean_rewards.append(mean_reward)

        # # Plot training progress
        # plt.plot(iterations, mean_rewards)
        # plt.xlabel('Training Iteration')
        # plt.ylabel('Mean Episode Reward (last 100 episodes)')
        # plt.title('Training Progress')
        # plt.grid()
        # plt.show()
    else:
        print("No valid checkpoint found. Unable to restore the model.")
except KeyError as e:
    print(f"KeyError encountered: {e}. Unable to retrieve the required metric.")
    if best_checkpoint is not None:
        print(f"Best checkpoint: {best_checkpoint}")

        # Use the trained model
        obs = env.reset()[0]
        trainer = PPO(config=config)
        trainer.restore(best_checkpoint)

        done = {agent: False for agent in env.agents}
        done['__all__'] = False

        should_render = True  # Change to True to visualize

        while not done['__all__']:
            actions = {agent: trainer.compute_single_action(obs[agent], policy_id="shared_policy") for agent in env.agents if not done[agent]}
            obs, rewards, done, truncateds, info = env.step(actions)
            
            if should_render:
                render_pygame(env)

            # Track rewards for visualization
            episode_reward = sum(rewards.values())
            rewards_history.append(episode_reward)
            iterations.append(len(iterations) + 1)
            mean_reward = np.mean(rewards_history)
            mean_rewards.append(mean_reward)

        # # Plot training progress
        # plt.plot(iterations, mean_rewards)
        # plt.xlabel('Training Iteration')
        # plt.ylabel('Mean Episode Reward (last 100 episodes)')
        # plt.title('Training Progress')
        # plt.grid()
        # plt.show()
    else:
        print("No valid checkpoint found. Unable to restore the model.")
else:
    print("No valid trials found. Unable to retrieve the best checkpoint.")

2024-11-26 21:06:25,961	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_envs_per_worker` has been deprecated. Use `AlgorithmConfig.num_envs_per_env_runner` instead. This will raise an error in the future!
(PPO pid=21948) 2024-11-26 21:06:38,437	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_envs_per_worker` has been deprecated. Use `AlgorithmConfig.num_envs_per_env_runner` instead. This will raise an error in the future!
(RolloutWorker pid=26688) 2024-11-26 21:06:53,717	DEBUG rollout_worker.py:1767 -- Creating policy for shared_policy
(RolloutWorker pid=26688) 2024-11-26 21:06:53,718	DEBUG catalog.py:705 -- Created preprocessor <ray.rllib.models.preprocessors.OneHotPreprocessor object at 0x0000022F8D307CB0>: Discrete(14) -> (14,)
(RolloutWorker pid=26688) 2024-11-26 21:06:53,735	INFO policy.py:1243 -- Policy (worker=1) running on CPU.
(RolloutWorker pid=26688) 2024-11-26 21:06:53,735	INFO torch_policy_v2.py:108 -- Found 0 visible cuda device

Available metrics: Index(['num_healthy_workers', 'num_in_flight_async_sample_reqs',
       'num_remote_worker_restarts', 'num_agent_steps_sampled',
       'num_agent_steps_trained', 'num_env_steps_sampled',
       'num_env_steps_trained', 'num_env_steps_sampled_this_iter',
       'num_env_steps_trained_this_iter',
       'num_env_steps_sampled_throughput_per_sec',
       ...
       'info/learner/shared_policy/learner_stats/grad_gnorm',
       'info/learner/shared_policy/learner_stats/cur_kl_coeff',
       'info/learner/shared_policy/learner_stats/cur_lr',
       'info/learner/shared_policy/learner_stats/total_loss',
       'info/learner/shared_policy/learner_stats/policy_loss',
       'info/learner/shared_policy/learner_stats/vf_loss',
       'info/learner/shared_policy/learner_stats/vf_explained_var',
       'info/learner/shared_policy/learner_stats/kl',
       'info/learner/shared_policy/learner_stats/entropy',
       'info/learner/shared_policy/learner_stats/entropy_coeff'],
      d

(RolloutWorker pid=15888) 2024-11-26 21:22:44,233	DEBUG rollout_worker.py:1767 -- Creating policy for shared_policy
(RolloutWorker pid=15888) 2024-11-26 21:22:44,234	DEBUG catalog.py:705 -- Created preprocessor <ray.rllib.models.preprocessors.OneHotPreprocessor object at 0x0000020F8D685A30>: Discrete(14) -> (14,)
(RolloutWorker pid=15888) 2024-11-26 21:22:44,247	INFO policy.py:1243 -- Policy (worker=3) running on CPU.
(RolloutWorker pid=15888) 2024-11-26 21:22:44,248	INFO torch_policy_v2.py:108 -- Found 0 visible cuda devices.
2024-11-26 21:22:48,361	INFO env_runner_group.py:314 -- Inferred observation/action spaces from remote worker (local worker has no env): {'shared_policy': (Discrete(14), Discrete(14)), '__env__': (Discrete(14), Discrete(14))}
2024-11-26 21:22:48,365	DEBUG rollout_worker.py:1767 -- Creating policy for shared_policy
2024-11-26 21:22:48,366	DEBUG catalog.py:705 -- Created preprocessor <ray.rllib.models.preprocessors.OneHotPreprocessor object at 0x000001C891565A30>: 

In [ ]:
import numpy as np
import networkx as nx
import gymnasium as gym
from gymnasium import spaces
import ray
from ray import tune
from ray.rllib.algorithms.ppo import PPO
from ray.rllib.utils.framework import try_import_torch
from ray.rllib.env.multi_agent_env import MultiAgentEnv
import sys
import torch
import os
from collections import deque
import time
import pygame
import xml.etree.ElementTree as ET
import re

# Set the CUDA_VISIBLE_DEVICES environment variable to the GPU index of the L4 (e.g., 0)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Replace "0" with the actual index of the L4 GPU based on `nvidia-smi`

pygame.init()
WIDTH, HEIGHT = 1200, 900
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Aircraft Movement on Taxiways")

# Parse the KML file
file_path = "5x5.kml"  # Update this path to your KML file
tree = ET.parse(file_path)
root = tree.getroot()
namespace = {'kml': 'http://www.opengis.net/kml/2.2'}

# Lists to store the taxiways and gates
taxiways = []
gates = []

# Extract LineStrings (taxiways and runways) and Points (gates)
for placemark in root.findall('.//kml:Placemark', namespace):
    line_string = placemark.find('.//kml:LineString/kml:coordinates', namespace)
    if line_string is not None:
        line_coords = line_string.text.strip().split()
        path = []
        for coord in line_coords:
            lon, lat, _ = map(float, coord.split(','))
            path.append((lon, lat))
        taxiway = {'path': path}
        name = placemark.find('kml:name', namespace)
        if name is not None:
            placemark_name = name.text.strip()
            taxiway['name'] = placemark_name
            match = re.search(r'\b(L1|L2|L3)\b', placemark_name)
            if match:
                taxiway['code'] = match.group(1)
            else:
                taxiway['code'] = ''
            if 'CENTRELINE' in placemark_name.upper():
                taxiway['is_runway'] = True
        taxiways.append(taxiway)
    point = placemark.find('.//kml:Point/kml:coordinates', namespace)
    if point is not None:
        coord = point.text.strip().split(',')
        lon, lat, _ = map(float, coord)
        gates.append({'position': (lon, lat)})

# Find the bounds of all coordinates
all_taxiway_coords = [coord for taxiway in taxiways for coord in taxiway['path']]
all_gate_coords = [gate['position'] for gate in gates]
all_coords = all_taxiway_coords + all_gate_coords

lon_min = min(coord[0] for coord in all_coords)
lon_max = max(coord[0] for coord in all_coords)
lat_min = min(coord[1] for coord in all_coords)
lat_max = max(coord[1] for coord in all_coords)

margin = 0.05
lon_range = lon_max - lon_min
lat_range = lat_max - lat_min
lon_min -= lon_range * margin
lon_max += lon_range * margin
lat_min -= lat_range * margin
lat_max += lat_range * margin

def convert_coords(lon, lat):
    x = (lon - lon_min) / (lon_max - lon_min) * WIDTH
    y = HEIGHT - (lat - lat_min) / (lat_max - lat_min) * HEIGHT
    return pygame.math.Vector2(x, y)

for taxiway in taxiways:
    pixel_path = [convert_coords(lon, lat) for lon, lat in taxiway['path']]
    taxiway['pixel_path'] = pixel_path

for gate in gates:
    gate['pixel_position'] = convert_coords(*gate['position'])

def distance(p1, p2):
    return (p1 - p2).length()

G = nx.Graph()
node_id_counter = 0
node_positions = {}
for idx, taxiway in enumerate(taxiways):
    path = taxiway['pixel_path']
    taxiway_name = taxiway.get('name', '')
    taxiway_code = taxiway.get('code', '')
    taxiway_nodes = []
    for i, point in enumerate(path):
        node_id = node_id_counter
        node_id_counter += 1
        G.add_node(node_id, pos=(point.x, point.y))
        G.nodes[node_id]['taxiway_name'] = taxiway_name
        G.nodes[node_id]['taxiway_code'] = taxiway_code
        if taxiway.get('is_runway'):
            G.nodes[node_id]['is_runway_node'] = True
        node_positions[node_id] = point
        taxiway_nodes.append(node_id)
        if i > 0:
            prev_node_id = taxiway_nodes[i - 1]
            G.add_edge(prev_node_id, node_id, weight=distance(point, node_positions[prev_node_id]))
    taxiway['node_ids'] = taxiway_nodes

def connect_taxiway_endpoints():
    CONNECTION_THRESHOLD = 30
    for taxiway1 in taxiways:
        for taxiway2 in taxiways:
            if taxiway1 == taxiway2:
                continue
            endpoints1 = [taxiway1['node_ids'][0], taxiway1['node_ids'][-1]]
            endpoints2 = [taxiway2['node_ids'][0], taxiway2['node_ids'][-1]]
            for node1 in endpoints1:
                for node2 in endpoints2:
                    pos1 = pygame.math.Vector2(G.nodes[node1]['pos'])
                    pos2 = pygame.math.Vector2(G.nodes[node2]['pos'])
                    if distance(pos1, pos2) < CONNECTION_THRESHOLD:
                        if not G.has_edge(node1, node2):
                            G.add_edge(node1, node2, weight=distance(pos1, pos2))

connect_taxiway_endpoints()

def connect_runways_to_taxiways():
    CONNECTION_THRESHOLD = 30
    for runway in [t for t in taxiways if t.get('is_runway')]:
        for taxiway in [t for t in taxiways if not t.get('is_runway')]:
            for runway_node in runway['node_ids']:
                for taxiway_node in taxiway['node_ids']:
                    pos_runway = pygame.math.Vector2(G.nodes[runway_node]['pos'])
                    pos_taxiway = pygame.math.Vector2(G.nodes[taxiway_node]['pos'])
                    if distance(pos_runway, pos_taxiway) < CONNECTION_THRESHOLD:
                        if not G.has_edge(runway_node, taxiway_node):
                            G.add_edge(runway_node, taxiway_node, weight=distance(pos_runway, pos_taxiway))

connect_runways_to_taxiways()

flipped_positions = {node: (pos[0], -pos[1]) for node, pos in node_positions.items()}

# Ensure all edges have a weight of 1
for edge in G.edges():
    G[edge[0]][edge[1]]['weight'] = 1

# Custom Multi-Agent Environment for Shortest Path Finding (RLlib-compatible)
class RoadIntersectionEnv(MultiAgentEnv):
    def __init__(self, graph, num_agents=3):
        super().__init__()
        self.graph = graph
        self.total_agents = num_agents
        self.agents = [f"car_{i}" for i in range(self.total_agents)]
        self.pos = {agent: None for agent in self.agents}
        self.distance_travelled = {agent: 0 for agent in self.agents}

        # Use Discrete spaces to represent each agent's current node in the graph
        self.observation_space = spaces.Discrete(len(self.graph.nodes))
        self.action_space = spaces.Discrete(len(self.graph.nodes))

    def reset(self, seed=None, options=None):
        if seed is not None:
            np.random.seed(seed)

        nodes = np.array(list(self.graph.nodes)).flatten()
        for agent in self.agents:
            self.pos[agent] = np.random.choice(nodes)
            self.distance_travelled[agent] = 0
        self.done_agents = set()
        obs = {agent: self.pos[agent] for agent in self.agents}  # Return observations as a dictionary
        return obs, {}

    def step(self, action_dict):
        rewards = {agent: 0 for agent in self.agents}
        dones = {agent: False for agent in self.agents}
        infos = {}
        truncateds = {agent: False for agent in self.agents}

        # Move each agent
        for agent, action in action_dict.items():
            if agent in self.done_agents:
                continue

            current_node = self.pos[agent]
            next_node = action

            # Ensure next_node is a valid neighbor of the current node
            if current_node in self.graph and next_node in self.graph[current_node]:
                self.pos[agent] = next_node
                self.distance_travelled[agent] += nx.get_edge_attributes(self.graph, 'weight').get((current_node, next_node), 1)
                rewards[agent] = -0.1  # Small negative reward to encourage shorter paths  # Encourage shorter paths

        # Detect collisions
        occupied_nodes = {}
        for agent in self.agents:
            if agent in self.done_agents:
                continue

            pos = self.pos[agent]
            if pos not in occupied_nodes:
                occupied_nodes[pos] = agent
            else:
                # Collision occurred
                rewards[agent] -= 5  # Reduced penalty for collision
                other_agent = occupied_nodes[pos]
                rewards[other_agent] -= 10
                dones[agent] = True
                dones[other_agent] = True
                self.done_agents.add(agent)
                self.done_agents.add(other_agent)

        # Update done flags for all agents
        for agent in self.agents:
            if agent not in self.done_agents:
                dones[agent] = False
            else:
                dones[agent] = True
                truncateds[agent] = True  # Mark as truncated to properly handle episode end

        # Check if all agents are done
        all_done = len(self.done_agents) == self.total_agents
        dones['__all__'] = all_done
        truncateds['__all__'] = all_done

        obs = {agent: self.pos[agent] for agent in self.agents if agent not in self.done_agents}  # Return observations as a dictionary
        infos = {agent: {} for agent in self.agents if agent not in self.done_agents}  # Update infos to match the current observation keys

        return obs, rewards, dones, truncateds, infos

def render_pygame(env):
    screen.fill((255, 255, 255))  # Fill background with white

    # Draw taxiways
    for taxiway in taxiways:
        pixel_path = taxiway['pixel_path']
        if len(pixel_path) > 1:
            pygame.draw.lines(screen, (0, 0, 0), False, [(point.x, point.y) for point in pixel_path], 2)

    # Draw nodes
    for node in env.graph.nodes:
        pos = env.graph.nodes[node]['pos']
        pygame.draw.circle(screen, (0, 0, 255), (int(pos[0]), int(pos[1])), 5)

    # Draw agents
    colors = [(255, 0, 0), (0, 255, 0), (128, 0, 128)]
    for idx, agent in enumerate(env.agents):
        if env.pos[agent] is not None:
            pos = env.graph.nodes[env.pos[agent]]['pos']
            pygame.draw.circle(screen, colors[idx % len(colors)], (int(pos[0]), int(pos[1])), 10)

    pygame.display.flip()  # Update the display

# Adjust training config and PPO code accordingly

# Create environment
env = RoadIntersectionEnv(G, num_agents=3)

# RLlib Training Configuration with fixed typo
ray.init(ignore_reinit_error=True, num_gpus=1, log_to_driver=True)

from ray.tune.registry import register_env

def env_creator(env_config):
    return RoadIntersectionEnv(env_config['graph'], env_config['num_agents'])

register_env("RoadIntersectionEnv", lambda env_config: env_creator(env_config))

def policy_mapping_fn(agent_id, *args, **kwargs):
    return "shared_policy"

config = {
    "rllib_version": "2.38",
    "env": "RoadIntersectionEnv",
    "env_config": {
        "graph": G,
        "num_agents": 3
    },
    "framework": "torch",  # Switch to PyTorch framework
    "num_workers": 7,  # Adjust number of workers to utilize available CPU cores
    "num_envs_per_worker": 80,  # Set environments per worker to balance CPU usage
    "num_gpus": 0,  # Specify that GPU should be used
    "train_batch_size": 8000,  # Increased batch size for more data per update
    "sgd_minibatch_size": 128,  # Increase minibatch size for training
    "lr": 1e-4,  # Reduced learning rate for stability
    "gamma": 0.99,  # Discount factor for reward
    "log_level": "DEBUG",
    "multiagent": {
        "policies": {
            "shared_policy": (None, env.observation_space, env.action_space, {})
        },
        "policy_mapping_fn": policy_mapping_fn,
        "policies_to_train": ["shared_policy"],
    },
    "exploration_config": {  # Add exploration configuration
        "type": "EpsilonGreedy",
        "initial_epsilon": 1.0,
        "final_epsilon": 0.05,
        "epsilon_timesteps": 20000  # Increased exploration time for better learning
    },
    "monitor": True,  # Enable monitoring for visualization
    "metric": "episode_reward_mean",
    "mode": "max"
}

# Track training progress
rewards_history = deque(maxlen=100)
iterations = []
mean_rewards = []

# Train the model using PPO with more frequent checkpoints
analysis = tune.run(
    "PPO", 
    config=config,
    reuse_actors=True, 
    stop={"training_iteration": 25}, 
    verbose=1, 
     storage_path="D:/aniru-windows/Capstone-Implementation/Localrun_ray_checkpoints",  
    checkpoint_at_end=True,
    checkpoint_freq=5  # Save checkpoints more frequently
)

# Save the model
try:
    print("Available metrics:", analysis.results_df.columns)
    if analysis.trials and analysis.get_best_trial(metric="training_iteration", mode="max") is not None:
        best_trial = analysis.get_best_trial(metric="training_iteration", mode="max")
        best_checkpoint = analysis.get_best_checkpoint(trial=best_trial, metric="training_iteration", mode="max")
    else:
        # Fallback if 'episode_reward_mean' is not available
        best_trial = analysis.get_best_trial(metric="training_iteration", mode="max")
        best_checkpoint = analysis.get_best_checkpoint(trial=best_trial, metric="training_iteration", mode="max")

    if best_checkpoint is not None:
        print(f"Best checkpoint: {best_checkpoint}")

        # Use the trained model
        obs = env.reset()[0]
        trainer = PPO(env="RoadIntersectionEnv", config=config)
        trainer.restore(best_checkpoint)

        done = {agent: False for agent in env.agents}
        done['__all__'] = False

        should_render = True  # Change to True to visualize

        while not done['__all__']:
            actions = {agent: trainer.compute_single_action(obs[agent], policy_id="shared_policy", explore=True) for agent in env.agents if not done[agent]}
            obs, rewards, done, truncateds, infos = env.step(actions)
            
            if should_render:
                render_pygame(env)

            # Track rewards for visualization
            episode_reward = sum(rewards.values())
            rewards_history.append(episode_reward)
            iterations.append(len(iterations) + 1)
            mean_reward = np.mean(rewards_history)
            mean_rewards.append(mean_reward)

        # # Plot training progress
        # plt.plot(iterations, mean_rewards)
        # plt.xlabel('Training Iteration')
        # plt.ylabel('Mean Episode Reward (last 100 episodes)')
        # plt.title('Training Progress')
        # plt.grid()
        # plt.show()
    else:
        print("No valid checkpoint found. Unable to restore the model.")
except KeyError as e:
    print(f"KeyError encountered: {e}. Unable to retrieve the required metric.")
    if best_checkpoint is not None:
        print(f"Best checkpoint: {best_checkpoint}")

        # Use the trained model
        obs = env.reset()[0]
        trainer = PPO(config=config)
        trainer.restore(best_checkpoint)

        done = {agent: False for agent in env.agents}
        done['__all__'] = False

        should_render = True  # Change to True to visualize

        while not done['__all__']:
            actions = {agent: trainer.compute_single_action(obs[agent], policy_id="shared_policy") for agent in env.agents if not done[agent]}
            obs, rewards, done, truncateds, info = env.step(actions)
            
            if should_render:
                render_pygame(env)

            # Track rewards for visualization
            episode_reward = sum(rewards.values())
            rewards_history.append(episode_reward)
            iterations.append(len(iterations) + 1)
            mean_reward = np.mean(rewards_history)
            mean_rewards.append(mean_reward)

        # # Plot training progress
        # plt.plot(iterations, mean_rewards)
        # plt.xlabel('Training Iteration')
        # plt.ylabel('Mean Episode Reward (last 100 episodes)')
        # plt.title('Training Progress')
        # plt.grid()
        # plt.show()
    else:
        print("No valid checkpoint found. Unable to restore the model.")
else:
    print("No valid trials found. Unable to retrieve the best checkpoint.")

2024-11-26 23:03:26,139	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_envs_per_worker` has been deprecated. Use `AlgorithmConfig.num_envs_per_env_runner` instead. This will raise an error in the future!
(PPO pid=30884) 2024-11-26 23:03:41,640	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_envs_per_worker` has been deprecated. Use `AlgorithmConfig.num_envs_per_env_runner` instead. This will raise an error in the future!
(RolloutWorker pid=28932) 2024-11-26 23:04:03,286	DEBUG rollout_worker.py:1767 -- Creating policy for shared_policy
(RolloutWorker pid=28932) 2024-11-26 23:04:03,287	DEBUG catalog.py:705 -- Created preprocessor <ray.rllib.models.preprocessors.OneHotPreprocessor object at 0x000001F80D7BC590>: Discrete(14) -> (14,)
(RolloutWorker pid=28932) 2024-11-26 23:04:03,314	INFO policy.py:1243 -- Policy (worker=2) running on CPU.
(RolloutWorker pid=28932) 2024-11-26 23:04:03,314	INFO torch_policy_v2.py:108 -- Found 0 visible cuda device

Available metrics: Index(['num_healthy_workers', 'num_in_flight_async_sample_reqs',
       'num_remote_worker_restarts', 'num_agent_steps_sampled',
       'num_agent_steps_trained', 'num_env_steps_sampled',
       'num_env_steps_trained', 'num_env_steps_sampled_this_iter',
       'num_env_steps_trained_this_iter',
       'num_env_steps_sampled_throughput_per_sec',
       ...
       'info/learner/shared_policy/learner_stats/grad_gnorm',
       'info/learner/shared_policy/learner_stats/cur_kl_coeff',
       'info/learner/shared_policy/learner_stats/cur_lr',
       'info/learner/shared_policy/learner_stats/total_loss',
       'info/learner/shared_policy/learner_stats/policy_loss',
       'info/learner/shared_policy/learner_stats/vf_loss',
       'info/learner/shared_policy/learner_stats/vf_explained_var',
       'info/learner/shared_policy/learner_stats/kl',
       'info/learner/shared_policy/learner_stats/entropy',
       'info/learner/shared_policy/learner_stats/entropy_coeff'],
      d

(RolloutWorker pid=33028) 2024-11-26 23:22:09,636	DEBUG rollout_worker.py:1767 -- Creating policy for shared_policy
(RolloutWorker pid=33028) 2024-11-26 23:22:09,637	DEBUG catalog.py:705 -- Created preprocessor <ray.rllib.models.preprocessors.OneHotPreprocessor object at 0x000002508D607C80>: Discrete(14) -> (14,)
(RolloutWorker pid=33028) 2024-11-26 23:22:09,653	INFO policy.py:1243 -- Policy (worker=2) running on CPU.
(RolloutWorker pid=33028) 2024-11-26 23:22:09,654	INFO torch_policy_v2.py:108 -- Found 0 visible cuda devices.
(RolloutWorker pid=33028) 2024-11-26 23:22:13,888	INFO util.py:118 -- Using connectors:
(RolloutWorker pid=33028) 2024-11-26 23:22:13,888	INFO util.py:119 --     AgentConnectorPipeline
(RolloutWorker pid=33028)         ObsPreprocessorConnector
(RolloutWorker pid=33028)         StateBufferConnector
(RolloutWorker pid=33028)         ViewRequirementAgentConnector
(RolloutWorker pid=33028) 2024-11-26 23:22:13,888	INFO util.py:120 --     ActionConnectorPipeline
(Rollo

# Training well, ray tune used, renders at end 

In [4]:
import numpy as np
import networkx as nx
import gymnasium as gym
from gymnasium import spaces
import ray
from ray import tune
from ray.rllib.algorithms.ppo import PPO
from ray.rllib.utils.framework import try_import_torch
from ray.rllib.env.multi_agent_env import MultiAgentEnv
import sys
import torch
import os
from collections import deque
import time
import pygame
import xml.etree.ElementTree as ET
import re

# Set the CUDA_VISIBLE_DEVICES environment variable to the GPU index of the L4 (e.g., 0)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Replace "0" with the actual index of the L4 GPU based on `nvidia-smi`

pygame.init()
WIDTH, HEIGHT = 1200, 900
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Aircraft Movement on Taxiways")

# Parse the KML file
file_path = "reduced_nodes.kml"  # Update this path to your KML file
tree = ET.parse(file_path)
root = tree.getroot()
namespace = {'kml': 'http://www.opengis.net/kml/2.2'}

# Lists to store the taxiways and gates
taxiways = []
gates = []

# Extract LineStrings (taxiways and runways) and Points (gates)
for placemark in root.findall('.//kml:Placemark', namespace):
    line_string = placemark.find('.//kml:LineString/kml:coordinates', namespace)
    if line_string is not None:
        line_coords = line_string.text.strip().split()
        path = []
        for coord in line_coords:
            lon, lat, _ = map(float, coord.split(','))
            path.append((lon, lat))
        taxiway = {'path': path}
        name = placemark.find('kml:name', namespace)
        if name is not None:
            placemark_name = name.text.strip()
            taxiway['name'] = placemark_name
            match = re.search(r'\b(L1|L2|L3)\b', placemark_name)
            if match:
                taxiway['code'] = match.group(1)
            else:
                taxiway['code'] = ''
            if 'CENTRELINE' in placemark_name.upper():
                taxiway['is_runway'] = True
        taxiways.append(taxiway)
    point = placemark.find('.//kml:Point/kml:coordinates', namespace)
    if point is not None:
        coord = point.text.strip().split(',')
        lon, lat, _ = map(float, coord)
        gates.append({'position': (lon, lat)})

# Find the bounds of all coordinates
all_taxiway_coords = [coord for taxiway in taxiways for coord in taxiway['path']]
all_gate_coords = [gate['position'] for gate in gates]
all_coords = all_taxiway_coords + all_gate_coords

lon_min = min(coord[0] for coord in all_coords)
lon_max = max(coord[0] for coord in all_coords)
lat_min = min(coord[1] for coord in all_coords)
lat_max = max(coord[1] for coord in all_coords)

margin = 0.05
lon_range = lon_max - lon_min
lat_range = lat_max - lat_min
lon_min -= lon_range * margin
lon_max += lon_range * margin
lat_min -= lat_range * margin
lat_max += lat_range * margin

def convert_coords(lon, lat):
    x = (lon - lon_min) / (lon_max - lon_min) * WIDTH
    y = HEIGHT - (lat - lat_min) / (lat_max - lat_min) * HEIGHT
    return pygame.math.Vector2(x, y)

for taxiway in taxiways:
    pixel_path = [convert_coords(lon, lat) for lon, lat in taxiway['path']]
    taxiway['pixel_path'] = pixel_path

for gate in gates:
    gate['pixel_position'] = convert_coords(*gate['position'])

def distance(p1, p2):
    return (p1 - p2).length()

G = nx.Graph()
node_id_counter = 0
node_positions = {}
for idx, taxiway in enumerate(taxiways):
    path = taxiway['pixel_path']
    taxiway_name = taxiway.get('name', '')
    taxiway_code = taxiway.get('code', '')
    taxiway_nodes = []
    for i, point in enumerate(path):
        node_id = node_id_counter
        node_id_counter += 1
        G.add_node(node_id, pos=(point.x, point.y))
        G.nodes[node_id]['taxiway_name'] = taxiway_name
        G.nodes[node_id]['taxiway_code'] = taxiway_code
        if taxiway.get('is_runway'):
            G.nodes[node_id]['is_runway_node'] = True
        node_positions[node_id] = point
        taxiway_nodes.append(node_id)
        if i > 0:
            prev_node_id = taxiway_nodes[i - 1]
            G.add_edge(prev_node_id, node_id, weight=distance(point, node_positions[prev_node_id]))
    taxiway['node_ids'] = taxiway_nodes

def connect_taxiway_endpoints():
    CONNECTION_THRESHOLD = 30
    for taxiway1 in taxiways:
        for taxiway2 in taxiways:
            if taxiway1 == taxiway2:
                continue
            endpoints1 = [taxiway1['node_ids'][0], taxiway1['node_ids'][-1]]
            endpoints2 = [taxiway2['node_ids'][0], taxiway2['node_ids'][-1]]
            for node1 in endpoints1:
                for node2 in endpoints2:
                    pos1 = pygame.math.Vector2(G.nodes[node1]['pos'])
                    pos2 = pygame.math.Vector2(G.nodes[node2]['pos'])
                    if distance(pos1, pos2) < CONNECTION_THRESHOLD:
                        if not G.has_edge(node1, node2):
                            G.add_edge(node1, node2, weight=distance(pos1, pos2))

connect_taxiway_endpoints()

def connect_runways_to_taxiways():
    CONNECTION_THRESHOLD = 30
    for runway in [t for t in taxiways if t.get('is_runway')]:
        for taxiway in [t for t in taxiways if not t.get('is_runway')]:
            for runway_node in runway['node_ids']:
                for taxiway_node in taxiway['node_ids']:
                    pos_runway = pygame.math.Vector2(G.nodes[runway_node]['pos'])
                    pos_taxiway = pygame.math.Vector2(G.nodes[taxiway_node]['pos'])
                    if distance(pos_runway, pos_taxiway) < CONNECTION_THRESHOLD:
                        if not G.has_edge(runway_node, taxiway_node):
                            G.add_edge(runway_node, taxiway_node, weight=distance(pos_runway, pos_taxiway))

connect_runways_to_taxiways()

flipped_positions = {node: (pos[0], -pos[1]) for node, pos in node_positions.items()}

# Ensure all edges have a weight of 1
for edge in G.edges():
    G[edge[0]][edge[1]]['weight'] = 1

# Predefined gates and runway endpoints for agents to start and end
gate_coordinates = [
    (77.71046165300096, 13.20355775032989),
    (77.71066072998956, 13.20166801620613),
    (77.70315360361252, 13.20165318995302),
    (77.71305689010458, 13.201637816132),
    (77.70195813939766, 13.20169406641297),
    (77.71149825860202, 13.20165032306983),
    (77.72279066770344, 13.19968338101175),
    (77.7132055646255, 13.20167917723071),
    (77.7089924860226, 13.20357466411958),
    (77.71648635375834, 13.20336974504588)
]

runway_endpoints = [
    (77.68686735208298, 13.20718308006773),
    (77.7222951686759, 13.20687066472886),
    (77.69060216489893, 13.18975920793089),
    (77.72642499411846, 13.18944676842937)
]

# Add nodes for gates and runway endpoints to the graph
for gate_coord in gate_coordinates:
    gate_pixel_position = convert_coords(*gate_coord)
    node_id = node_id_counter
    node_id_counter += 1
    G.add_node(node_id, pos=(gate_pixel_position.x, gate_pixel_position.y), is_gate=True)
    node_positions[node_id] = gate_pixel_position

for runway_coord in runway_endpoints:
    runway_pixel_position = convert_coords(*runway_coord)
    node_id = node_id_counter
    node_id_counter += 1
    G.add_node(node_id, pos=(runway_pixel_position.x, runway_pixel_position.y), is_runway_endpoint=True)
    node_positions[node_id] = runway_pixel_position

# Custom Multi-Agent Environment for Shortest Path Finding (RLlib-compatible)
class RoadIntersectionEnv(MultiAgentEnv):
    def __init__(self, graph, num_agents=3):
        super().__init__()
        self.graph = graph
        self.total_agents = num_agents
        self.agents = [f"car_{i}" for i in range(self.total_agents)]
        self.pos = {agent: None for agent in self.agents}
        self.distance_travelled = {agent: 0 for agent in self.agents}

        # Use Discrete spaces to represent each agent's current node in the graph
        self.observation_space = spaces.Discrete(len(self.graph.nodes))
        self.action_space = spaces.Discrete(len(self.graph.nodes))

    def reset(self, seed=None, options=None):
        if seed is not None:
            np.random.seed(seed)

        # Randomly assign starting positions from gates and runway endpoints
        available_nodes = [node for node, data in self.graph.nodes(data=True) if data.get('is_gate') or data.get('is_runway_endpoint')]
        for agent in self.agents:
            self.pos[agent] = np.random.choice(available_nodes)
            self.distance_travelled[agent] = 0
        self.done_agents = set()
        obs = {agent: self.pos[agent] for agent in self.agents}  # Return observations as a dictionary
        return obs, {}

    def step(self, action_dict):
        rewards = {agent: 0 for agent in self.agents}
        dones = {agent: False for agent in self.agents}
        infos = {}
        truncateds = {agent: False for agent in self.agents}

        # Move each agent
        for agent, action in action_dict.items():
            if agent in self.done_agents:
                continue

            current_node = self.pos[agent]
            next_node = action

            # Ensure next_node is a valid neighbor of the current node
            if current_node in self.graph and next_node in self.graph[current_node]:
                self.pos[agent] = next_node
                self.distance_travelled[agent] += nx.get_edge_attributes(self.graph, 'weight').get((current_node, next_node), 1)
                rewards[agent] = -0.1  # Small negative reward to encourage shorter paths  # Encourage shorter paths

        # Detect collisions
        occupied_nodes = {}
        for agent in self.agents:
            if agent in self.done_agents:
                continue

            pos = self.pos[agent]
            if pos not in occupied_nodes:
                occupied_nodes[pos] = agent
            else:
                # Collision occurred
                rewards[agent] -= 5  # Reduced penalty for collision
                other_agent = occupied_nodes[pos]
                rewards[other_agent] -= 10
                dones[agent] = True
                dones[other_agent] = True
                self.done_agents.add(agent)
                self.done_agents.add(other_agent)

        # Update done flags for all agents
        for agent in self.agents:
            if agent not in self.done_agents:
                dones[agent] = False
            else:
                dones[agent] = True
                truncateds[agent] = True  # Mark as truncated to properly handle episode end

        # Check if all agents are done
        all_done = len(self.done_agents) == self.total_agents
        dones['__all__'] = all_done
        truncateds['__all__'] = all_done

        obs = {agent: self.pos[agent] for agent in self.agents if agent not in self.done_agents}  # Return observations as a dictionary
        infos = {agent: {} for agent in self.agents if agent not in self.done_agents}  # Update infos to match the current observation keys

        return obs, rewards, dones, truncateds, infos

def render_pygame(env):
    screen.fill((255, 255, 255))  # Fill background with white

    # Draw taxiways
    for taxiway in taxiways:
        pixel_path = taxiway['pixel_path']
        if len(pixel_path) > 1:
            pygame.draw.lines(screen, (0, 0, 0), False, [(point.x, point.y) for point in pixel_path], 2)

    # Draw nodes
    for node in env.graph.nodes:
        pos = env.graph.nodes[node]['pos']
        pygame.draw.circle(screen, (0, 0, 255), (int(pos[0]), int(pos[1])), 5)

    # Draw agents
    colors = [(255, 0, 0), (0, 255, 0), (128, 0, 128)]
    for idx, agent in enumerate(env.agents):
        if env.pos[agent] is not None:
            pos = env.graph.nodes[env.pos[agent]]['pos']
            pygame.draw.circle(screen, colors[idx % len(colors)], (int(pos[0]), int(pos[1])), 10)

    pygame.display.flip()  # Update the display

# Adjust training config and PPO code accordingly

# Create environment
env = RoadIntersectionEnv(G, num_agents=3)

# RLlib Training Configuration with fixed typo
ray.init(ignore_reinit_error=True, num_gpus=1, log_to_driver=True)

from ray.tune.registry import register_env

def env_creator(env_config):
    return RoadIntersectionEnv(env_config['graph'], env_config['num_agents'])

register_env("RoadIntersectionEnv", lambda env_config: env_creator(env_config))

def policy_mapping_fn(agent_id, *args, **kwargs):
    return "shared_policy"

config = {
    "rllib_version": "2.38",
    "env": "RoadIntersectionEnv",
    "env_config": {
        "graph": G,
        "num_agents": 3
    },
    "framework": "torch",  # Switch to PyTorch framework
    "num_workers": 8,  # Adjust number of workers to utilize available CPU cores
    "num_envs_per_worker": 8,  # Set environments per worker to balance CPU usage
    "num_gpus": 1,  # Specify that GPU should be used
    "train_batch_size": 8000,  # Increased batch size for more data per update
    "sgd_minibatch_size": 128,  # Increase minibatch size for training
    "lr": 1e-2,  # Reduced learning rate for stability
    "gamma": 0.5,  # Discount factor for reward
    "log_level": "WARN",
    "multiagent": {
        "policies": {
            "shared_policy": (None, env.observation_space, env.action_space, {})
        },
        "policy_mapping_fn": policy_mapping_fn,
        "policies_to_train": ["shared_policy"],
    },
   
    "monitor": True,  # Enable monitoring for visualization
    "metric": "episode_reward_mean",
    "mode": "max"
}

# Track training progress
rewards_history = deque(maxlen=100)
iterations = []
mean_rewards = []

# Train the model using PPO with more frequent checkpoints
analysis = tune.run(
    "PPO", 
    config=config,
    reuse_actors=True, 
    stop={"training_iteration": 5}, 
    verbose=1, 
    storage_path="D:/aniru-windows/Capstone-Implementation/Localrun_ray_checkpoints/", 
    checkpoint_at_end=True,
    checkpoint_freq=1  # Save checkpoints more frequently
)

# Save the model
try:
    print("Available metrics:", analysis.results_df.columns)
    if analysis.trials and analysis.get_best_trial(metric="training_iteration", mode="max") is not None:
        best_trial = analysis.get_best_trial(metric="training_iteration", mode="max")
        best_checkpoint = analysis.get_best_checkpoint(trial=best_trial, metric="training_iteration", mode="max")
    else:
        # Fallback if 'episode_reward_mean' is not available
        best_trial = analysis.get_best_trial(metric="training_iteration", mode="max")
        best_checkpoint = analysis.get_best_checkpoint(trial=best_trial, metric="training_iteration", mode="max")

    if best_checkpoint is not None:
        print(f"Best checkpoint: {best_checkpoint}")

        # Use the trained model
        obs = env.reset()[0]
        trainer = PPO(env="RoadIntersectionEnv", config=config)
        trainer.restore(best_checkpoint)

        done = {agent: False for agent in env.agents}
        done['__all__'] = False

        should_render = True  # Change to True to visualize

        while not done['__all__']:
            actions = {agent: trainer.compute_single_action(obs[agent], policy_id="shared_policy", explore=True) for agent in env.agents if not done[agent]}
            obs, rewards, done, truncateds, infos = env.step(actions)
            
            if should_render:
                render_pygame(env)

            # Track rewards for visualization
            episode_reward = sum(rewards.values())
            rewards_history.append(episode_reward)
            iterations.append(len(iterations) + 1)
            mean_reward = np.mean(rewards_history)
            mean_rewards.append(mean_reward)

        # # Plot training progress
        # plt.plot(iterations, mean_rewards)
        # plt.xlabel('Training Iteration')
        # plt.ylabel('Mean Episode Reward (last 100 episodes)')
        # plt.title('Training Progress')
        # plt.grid()
        # plt.show()
    else:
        print("No valid checkpoint found. Unable to restore the model.")
except KeyError as e:
    print(f"KeyError encountered: {e}. Unable to retrieve the required metric.")
    if best_checkpoint is not None:
        print(f"Best checkpoint: {best_checkpoint}")

        # Use the trained model
        obs = env.reset()[0]
        trainer = PPO(config=config)
        trainer.restore(best_checkpoint)

        done = {agent: False for agent in env.agents}
        done['__all__'] = False

        should_render = True  # Change to True to visualize

        while not done['__all__']:
            actions = {agent: trainer.compute_single_action(obs[agent], policy_id="shared_policy") for agent in env.agents if not done[agent]}
            obs, rewards, done, truncateds, info = env.step(actions)
            
            if should_render:
                render_pygame(env)

            # Track rewards for visualization
            episode_reward = sum(rewards.values())
            rewards_history.append(episode_reward)
            iterations.append(len(iterations) + 1)
            mean_reward = np.mean(rewards_history)
            mean_rewards.append(mean_reward)

        # # Plot training progress
        # plt.plot(iterations, mean_rewards)
        # plt.xlabel('Training Iteration')
        # plt.ylabel('Mean Episode Reward (last 100 episodes)')
        # plt.title('Training Progress')
        # plt.grid()
        # plt.show()
    else:
        print("No valid checkpoint found. Unable to restore the model.")
else:
    print("No valid trials found. Unable to retrieve the best checkpoint.")

2024-11-28 23:59:46,608	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_envs_per_worker` has been deprecated. Use `AlgorithmConfig.num_envs_per_env_runner` instead. This will raise an error in the future!
(PPO pid=33084) 2024-11-29 00:00:02,064	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_envs_per_worker` has been deprecated. Use `AlgorithmConfig.num_envs_per_env_runner` instead. This will raise an error in the future!
2024-11-29 00:00:21,528	WARNING tune.py:219 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2024-11-29 00:00:21,556	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to 'D:/aniru-windows/Capstone-Implementation/Localrun_ray_checkpoints/PPO_2024-11-28_23-59-46' in 0.0257s.
2024-11-29 00:00:31,579	INFO tune.py:1041 -- Total run time: 45.25 seconds (3

Available metrics: Index([], dtype='object')
No valid checkpoint found. Unable to restore the model.
No valid trials found. Unable to retrieve the best checkpoint.


# Code w/o ray tune, live training plotting apparently 

In [1]:
import numpy as np
import networkx as nx
import gymnasium as gym
from gymnasium import spaces
import ray
from ray import tune
from ray.rllib.algorithms.ppo import PPO, PPOConfig
from ray.rllib.env.multi_agent_env import MultiAgentEnv
import sys
import os
import pygame
import xml.etree.ElementTree as ET
import re
from collections import deque
import matplotlib.pyplot as plt
import time

# ================================
# Define the Distance Function at the Module Level
# ================================
def distance(p1, p2):
    return (p1 - p2).length()

# ================================
# Set Screen Dimensions (No Pygame Initialization Here)
# ================================
WIDTH, HEIGHT = 1200, 900

# ================================
# Parse the KML File
# ================================
def parse_kml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    namespace = {'kml': 'http://www.opengis.net/kml/2.2'}

    taxiways = []
    gates = []

    for placemark in root.findall('.//kml:Placemark', namespace):
        # Extract LineStrings (taxiways and runways)
        line_string = placemark.find('.//kml:LineString/kml:coordinates', namespace)
        if line_string is not None:
            line_coords = line_string.text.strip().split()
            path = []
            for coord in line_coords:
                lon, lat, _ = map(float, coord.split(','))
                path.append((lon, lat))
            taxiway = {'path': path}
            name = placemark.find('kml:name', namespace)
            if name is not None:
                placemark_name = name.text.strip()
                taxiway['name'] = placemark_name
                match = re.search(r'\b(L1|L2|L3)\b', placemark_name)
                if match:
                    taxiway['code'] = match.group(1)
                else:
                    taxiway['code'] = ''
                if 'CENTRELINE' in placemark_name.upper():
                    taxiway['is_runway'] = True
            taxiways.append(taxiway)

        # Extract Points (gates)
        point = placemark.find('.//kml:Point/kml:coordinates', namespace)
        if point is not None:
            coord = point.text.strip().split(',')
            lon, lat, _ = map(float, coord)
            gates.append({'position': (lon, lat)})

    return taxiways, gates

# ================================
# Build the NetworkX Graph
# ================================
def build_graph(taxiways, gates, width, height):
    all_taxiway_coords = [coord for taxiway in taxiways for coord in taxiway['path']]
    all_gate_coords = [gate['position'] for gate in gates]
    all_coords = all_taxiway_coords + all_gate_coords

    lon_min = min(coord[0] for coord in all_coords)
    lon_max = max(coord[0] for coord in all_coords)
    lat_min = min(coord[1] for coord in all_coords)
    lat_max = max(coord[1] for coord in all_coords)

    margin = 0.05
    lon_range = lon_max - lon_min
    lat_range = lat_max - lat_min
    lon_min -= lon_range * margin
    lon_max += lon_range * margin
    lat_min -= lat_range * margin
    lat_max += lat_range * margin

    def convert_coords(lon, lat):
        x = (lon - lon_min) / (lon_max - lon_min) * width
        y = height - (lat - lat_min) / (lat_max - lat_min) * height
        return pygame.math.Vector2(x, y)

    for taxiway in taxiways:
        pixel_path = [convert_coords(lon, lat) for lon, lat in taxiway['path']]
        taxiway['pixel_path'] = pixel_path

    for gate in gates:
        gate['pixel_position'] = convert_coords(*gate['position'])

    G = nx.Graph()
    node_id_counter = 0
    node_positions = {}
    for idx, taxiway in enumerate(taxiways):
        path = taxiway['pixel_path']
        taxiway_name = taxiway.get('name', '')
        taxiway_code = taxiway.get('code', '')
        taxiway_nodes = []
        for i, point in enumerate(path):
            node_id = node_id_counter
            node_id_counter += 1
            G.add_node(node_id, pos=(point.x, point.y))
            G.nodes[node_id]['taxiway_name'] = taxiway_name
            G.nodes[node_id]['taxiway_code'] = taxiway_code
            if taxiway.get('is_runway'):
                G.nodes[node_id]['is_runway_node'] = True
            node_positions[node_id] = point
            taxiway_nodes.append(node_id)
            if i > 0:
                prev_node_id = taxiway_nodes[i - 1]
                G.add_edge(prev_node_id, node_id, weight=distance(point, node_positions[prev_node_id]))
        taxiway['node_ids'] = taxiway_nodes

    def connect_taxiway_endpoints(G, taxiways):
        CONNECTION_THRESHOLD = 30
        for taxiway1 in taxiways:
            for taxiway2 in taxiways:
                if taxiway1 == taxiway2:
                    continue
                endpoints1 = [taxiway1['node_ids'][0], taxiway1['node_ids'][-1]]
                endpoints2 = [taxiway2['node_ids'][0], taxiway2['node_ids'][-1]]
                for node1 in endpoints1:
                    for node2 in endpoints2:
                        pos1 = pygame.math.Vector2(G.nodes[node1]['pos'])
                        pos2 = pygame.math.Vector2(G.nodes[node2]['pos'])
                        if distance(pos1, pos2) < CONNECTION_THRESHOLD:
                            if not G.has_edge(node1, node2):
                                G.add_edge(node1, node2, weight=distance(pos1, pos2))

    def connect_runways_to_taxiways(G, taxiways):
        CONNECTION_THRESHOLD = 30
        for runway in [t for t in taxiways if t.get('is_runway')]:
            for taxiway in [t for t in taxiways if not t.get('is_runway')]:
                for runway_node in runway['node_ids']:
                    for taxiway_node in taxiway['node_ids']:
                        pos_runway = pygame.math.Vector2(G.nodes[runway_node]['pos'])
                        pos_taxiway = pygame.math.Vector2(G.nodes[taxiway_node]['pos'])
                        if distance(pos_runway, pos_taxiway) < CONNECTION_THRESHOLD:
                            if not G.has_edge(runway_node, taxiway_node):
                                G.add_edge(runway_node, taxiway_node, weight=distance(pos_runway, pos_taxiway))

    connect_taxiway_endpoints(G, taxiways)
    connect_runways_to_taxiways(G, taxiways)

    flipped_positions = {node: (pos[0], -pos[1]) for node, pos in node_positions.items()}

    # Ensure all edges have a weight of 1
    for edge in G.edges():
        G[edge[0]][edge[1]]['weight'] = 1

    return G, node_positions, convert_coords

# ================================
# Add Predefined Gates and Runway Endpoints
# ================================
def add_predefined_nodes(G, node_positions, gate_coordinates, runway_endpoints, convert_coords, node_id_counter):
    for gate_coord in gate_coordinates:
        gate_pixel_position = convert_coords(*gate_coord)
        node_id = node_id_counter
        node_id_counter += 1
        G.add_node(node_id, pos=(gate_pixel_position.x, gate_pixel_position.y), is_gate=True)
        node_positions[node_id] = gate_pixel_position

    for runway_coord in runway_endpoints:
        runway_pixel_position = convert_coords(*runway_coord)
        node_id = node_id_counter
        node_id_counter += 1
        G.add_node(node_id, pos=(runway_pixel_position.x, runway_pixel_position.y), is_runway_endpoint=True)
        node_positions[node_id] = runway_pixel_position

    return G, node_positions, node_id_counter

# ================================
# Custom Multi-Agent Environment
# ================================
class RoadIntersectionEnv(MultiAgentEnv):
    def __init__(self, config):
        super().__init__()
        self.graph = config["graph"]
        self.total_agents = config["num_agents"]
        self.agents = [f"car_{i}" for i in range(self.total_agents)]
        self.pos = {agent: None for agent in self.agents}
        self.distance_travelled = {agent: 0 for agent in self.agents}

        # Use Discrete spaces to represent each agent's current node in the graph
        self.observation_space = spaces.Discrete(len(self.graph.nodes))
        self.action_space = spaces.Discrete(len(self.graph.nodes))

    def reset(self, seed=None, options=None):
        if seed is not None:
            np.random.seed(seed)

        # Randomly assign starting positions from gates and runway endpoints
        available_nodes = [node for node, data in self.graph.nodes(data=True) if data.get('is_gate') or data.get('is_runway_endpoint')]
        for agent in self.agents:
            self.pos[agent] = np.random.choice(available_nodes)
            self.distance_travelled[agent] = 0
        self.done_agents = set()
        obs = {agent: self.pos[agent] for agent in self.agents}  # Return observations as a dictionary
        return obs, {}

    def step(self, action_dict):
        rewards = {agent: 0 for agent in self.agents}
        dones = {agent: False for agent in self.agents}
        infos = {}
        truncateds = {agent: False for agent in self.agents}

        # Move each agent
        for agent, action in action_dict.items():
            if agent in self.done_agents:
                continue

            current_node = self.pos[agent]
            next_node = action

            # Ensure next_node is a valid neighbor of the current node
            if current_node in self.graph and next_node in self.graph[current_node]:
                self.pos[agent] = next_node
                self.distance_travelled[agent] += self.graph[current_node][next_node]['weight']
                rewards[agent] = -0.1  # Small negative reward to encourage shorter paths
            else:
                # Invalid action, penalize the agent
                rewards[agent] -= 1

        # Detect collisions
        occupied_nodes = {}
        for agent in self.agents:
            if agent in self.done_agents:
                continue

            pos = self.pos[agent]
            if pos not in occupied_nodes:
                occupied_nodes[pos] = agent
            else:
                # Collision occurred
                rewards[agent] -= 5  # Penalty for collision
                other_agent = occupied_nodes[pos]
                rewards[other_agent] -= 5
                dones[agent] = True
                dones[other_agent] = True
                self.done_agents.add(agent)
                self.done_agents.add(other_agent)

        # Update done flags for all agents
        for agent in self.agents:
            if agent not in self.done_agents:
                dones[agent] = False
            else:
                dones[agent] = True
                truncateds[agent] = True  # Mark as truncated to properly handle episode end

        # Check if all agents are done
        all_done = len(self.done_agents) == self.total_agents
        dones['__all__'] = all_done
        truncateds['__all__'] = all_done

        obs = {agent: self.pos[agent] for agent in self.agents if agent not in self.done_agents}  # Return observations as a dictionary
        infos = {agent: {} for agent in self.agents if agent not in self.done_agents}  # Update infos to match the current observation keys

        return obs, rewards, dones, truncateds, infos

    def render(self, mode='human'):
        # Get the current screen
        screen = pygame.display.get_surface()
        if screen is None:
            return

        screen.fill((255, 255, 255))  # Fill background with white

        # Draw taxiways
        for taxiway in taxiways:
            pixel_path = taxiway['pixel_path']
            if len(pixel_path) > 1:
                pygame.draw.lines(screen, (0, 0, 0), False, [(point.x, point.y) for point in pixel_path], 2)

        # Draw nodes
        for node in self.graph.nodes:
            pos = self.graph.nodes[node]['pos']
            pygame.draw.circle(screen, (0, 0, 255), (int(pos[0]), int(pos[1])), 5)

        # Draw agents
        colors = [(255, 0, 0), (0, 255, 0), (128, 0, 128)]
        for idx, agent in enumerate(self.agents):
            if self.pos[agent] is not None:
                pos = self.graph.nodes[self.pos[agent]]['pos']
                pygame.draw.circle(screen, colors[idx % len(colors)], (int(pos[0]), int(pos[1])), 10)

        pygame.display.flip()  # Update the display

# ================================
# Main Function
# ================================
def main():
    # Include global variables required by the environment
    global taxiways

    # ================================
    # Parse KML and Build Graph
    # ================================
    file_path = "reduced_nodes.kml"  # Update this path to your KML file
    taxiways, gates = parse_kml(file_path)
    G, node_positions, convert_coords = build_graph(taxiways, gates, WIDTH, HEIGHT)

    # Predefined gates and runway endpoints for agents to start and end
    gate_coordinates = [
        (77.71046165300096, 13.20355775032989),
        (77.71066072998956, 13.20166801620613),
        (77.70315360361252, 13.20165318995302),
        (77.71305689010458, 13.201637816132),
        (77.70195813939766, 13.20169406641297),
        (77.71149825860202, 13.20165032306983),
        (77.72279066770344, 13.19968338101175),
        (77.7132055646255, 13.20167917723071),
        (77.7089924860226, 13.20357466411958),
        (77.71648635375834, 13.20336974504588)
    ]

    runway_endpoints = [
        (77.68686735208298, 13.20718308006773),
        (77.7222951686759, 13.20687066472886),
        (77.69060216489893, 13.18975920793089),
        (77.72642499411846, 13.18944676842937)
    ]

    # Add predefined gates and runway endpoints to the graph
    node_id_counter = len(G.nodes())
    G, node_positions, node_id_counter = add_predefined_nodes(
        G, node_positions, gate_coordinates, runway_endpoints,
        convert_coords=convert_coords,
        node_id_counter=node_id_counter
    )

    # ================================
    # Initialize Ray
    # ================================
    ray.init(ignore_reinit_error=True, log_to_driver=True)

    # ================================
    # Register the Environment
    # ================================
    from ray.tune.registry import register_env

    def env_creator(env_config):
        return RoadIntersectionEnv(env_config)

    register_env("RoadIntersectionEnv", lambda env_config: env_creator(env_config))

    # ================================
    # Define Policy Mapping Function
    # ================================
    def policy_mapping_fn(agent_id, *args, **kwargs):
        return "shared_policy"

    # ================================
    # RLlib Training Configuration
    # ================================
    temp_env = RoadIntersectionEnv({"graph": G, "num_agents": 3})

    config = {
        "env": "RoadIntersectionEnv",
        "env_config": {
            "graph": G,
            "num_agents": 3
        },
        "framework": "torch",
        "num_workers": 0,  # Run in the main thread for rendering
        "num_envs_per_worker": 1,
        "num_gpus": 0,  # Set to 1 if you have a GPU and want to use it
        "train_batch_size": 4000,
        "sgd_minibatch_size": 128,
        "lr": 1e-3,
        "gamma": 0.99,
        "log_level": "WARN",
        "multiagent": {
            "policies": {
                "shared_policy": (None, temp_env.observation_space, temp_env.action_space, {})
            },
            "policy_mapping_fn": policy_mapping_fn,
            "policies_to_train": ["shared_policy"],
        },
    }

    # ================================
    # Initialize Pygame (Moved Here)
    # ================================
    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Aircraft Movement on Taxiways")

    # ================================
    # Create the PPO Trainer
    # ================================
    ppo_config = PPOConfig().environment(
        env="RoadIntersectionEnv",
        env_config=config["env_config"],
    ).framework(
        framework="torch"
    ).training(
        lr=config["lr"],
        train_batch_size=config["train_batch_size"],
        gamma=config["gamma"],
    ).resources(
        num_gpus=config["num_gpus"],
        num_cpus_per_worker=1,
    ).rollouts(
        num_rollout_workers=config["num_workers"],
        num_envs_per_worker=config["num_envs_per_worker"],
    ).multi_agent(
        policies=config["multiagent"]["policies"],
        policy_mapping_fn=policy_mapping_fn,
        policies_to_train=["shared_policy"],
    )

    # Set sgd_minibatch_size directly
    ppo_config.sgd_minibatch_size = config["sgd_minibatch_size"]

    # Build the trainer
    trainer = ppo_config.build()

    # ================================
    # Training Loop with Live Rendering and Checkpoints
    # ================================
    checkpoint_dir = "./checkpoints"
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

    max_iterations = 50  # Set the number of training iterations
    rewards_history = deque(maxlen=100)
    iterations_list = []
    mean_rewards = []

    try:
        for i in range(max_iterations):
            print(f"Starting training iteration {i+1}/{max_iterations}")
            
            # Perform one training iteration
            result = trainer.train()
            mean_reward = result.get('episode_reward_mean', 0)
            print(f"Iteration {i+1} results: {mean_reward} mean reward")

            # Append results for plotting
            iterations_list.append(i+1)
            mean_rewards.append(mean_reward)
            rewards_history.append(mean_reward)

            # Save checkpoint after each iteration
            checkpoint_path = trainer.save(checkpoint_dir)
            print(f"Checkpoint saved at {checkpoint_path}")

            # Reset the environment
            env = RoadIntersectionEnv(config["env_config"])
            obs, _ = env.reset()

            done = {agent: False for agent in env.agents}
            done['__all__'] = False

            should_render = True  # Set to True to visualize

            # Run one episode to visualize the agent's behavior
            while not done['__all__']:
                # Handle pygame events
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        ray.shutdown()
                        sys.exit()

                actions = {}
                for agent_id in obs:
                    action = trainer.compute_single_action(obs[agent_id], policy_id="shared_policy", explore=False)
                    actions[agent_id] = action

                obs, rewards, done, truncateds, infos = env.step(actions)

                if should_render:
                    env.render()

                # Optional: Add a delay for visualization purposes
                pygame.time.delay(100)  # Delay in milliseconds

        # ================================
        # Plot Training Progress
        # ================================
        plt.figure(figsize=(10, 6))
        plt.plot(iterations_list, mean_rewards, label='Mean Episode Reward')
        plt.xlabel('Training Iteration')
        plt.ylabel('Mean Episode Reward')
        plt.title('Training Progress')
        plt.legend()
        plt.grid()
        plt.show()

    except KeyboardInterrupt:
        print("Training interrupted by user.")

    finally:
        # ================================
        # Clean up Resources
        # ================================
        ray.shutdown()
        pygame.quit()

# ================================
# Run the Main Function
# ================================
if __name__ == "__main__":
    main()

2024-11-29 19:57:36,955	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-11-29 19:57:37,969	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-11-29 19:57:55,041	INFO worker.py:1807 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
2024-11-29 19:58:00,612	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.resources(num_cpus_per_worker)` has been deprecated. Use `AlgorithmConfig.env_runners(num_cpus_per_env_runner)` instead. This will raise an error in the future!
2024-11-29 19:58:00,613	WARNING deprecation.py:50 -- DeprecationWarning: `rollouts` has been deprecated. Use `AlgorithmConfig.env_runners(..)` instead. This will raise an error in the future!
2024-11-29 19:58:00,614	WARNING

Starting training iteration 1/50


2024-11-29 19:58:22,364	WARNING deprecation.py:50 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!


Iteration 1 results: 0 mean reward
Checkpoint saved at TrainingResult(checkpoint=Checkpoint(filesystem=local, path=./checkpoints), metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'shared_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 0.15109199859426484, 'cur_kl_coeff': 0.2, 'cur_lr': 0.0010000000000000002, 'total_loss': 9.908783388137817, 'policy_loss': -0.09068131533373767, 'vf_loss': 9.987498008443954, 'vf_explained_var': -0.00025003529186789873, 'kl': 0.05983342881704626, 'entropy': 7.801557375353279, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 127.65957446808511, 'num_grad_updates_lifetime': 1410.5, 'diff_num_grad_updates_vs_sampler_policy': 1409.5}}, 'num_env_steps_sampled': 4000, 'num_env_steps_trained': 4000, 'num_agent_steps_sampled': 12000, 'num_agent_steps_trained': 12000}, 'env_runners': {'episode_reward_max': nan, 'episode_reward_min': nan, 'episode_reward_mean': nan, 'episode_len_mean':

# Matplotlib rendering

In [7]:
import numpy as np
import networkx as nx
import gymnasium as gym
from gymnasium import spaces
import ray
from ray import tune
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from ray.tune import Callback
import sys
import os
import xml.etree.ElementTree as ET
import re
import matplotlib
import matplotlib.pyplot as plt
import time

# Set matplotlib backend to ensure compatibility with interactive plots
matplotlib.use('TkAgg')

# Custom Callback for Real-Time Plotting
class RealTimePlottingCallback(Callback):
    def __init__(self):
        self.iterations = []
        self.mean_rewards = []
        self.fig, (self.ax1, self.ax2) = plt.subplots(1, 2, figsize=(12, 6))
        plt.ion()  # Interactive mode for real-time plotting
        plt.show()
        self.current_positions = None
        self.graph = None

    def on_trial_result(self, iteration, trials, trial, result, **info):
        mean_reward = result.get("episode_reward_mean", 0)
        self.iterations.append(iteration)
        self.mean_rewards.append(mean_reward)

        # Update the reward plot
        # Update reward plot without clearing the entire axis
        self.ax1.lines = []  # Clear only the lines in the plot
        self.ax1.plot(self.iterations, self.mean_rewards, label="Mean Episode Reward")
        self.ax1.set_xlabel("Training Iteration")
        self.ax1.set_ylabel("Mean Episode Reward")
        self.ax1.set_title("Training Progress")
        self.ax1.legend()
        self.ax1.grid()

        # Update the agent movement plot if graph and positions are available
        if self.graph and self.current_positions:
            # Update agent movement plot without clearing the entire axis
            pos = nx.get_node_attributes(self.graph, 'pos')
            self.ax2.collections = []  # Clear existing nodes and edges       
            nx.draw(self.graph, pos, ax=self.ax2, node_color='lightblue', edge_color='gray', with_labels=False, node_size=20)
            agent_positions = np.array(list(self.current_positions.values()))
            self.ax2.scatter(agent_positions[:, 0], agent_positions[:, 1], color='red', s=100, label="Agents")
            self.ax2.set_title("Agent Movement in Environment")
            self.ax2.legend()

        plt.pause(0.01)  # Pause to allow the plot to update

    def update_positions(self, graph, current_positions):
        self.graph = graph
        self.current_positions = current_positions

# ================================
# Define the Distance Function at the Module Level
# ================================
def distance(p1, p2):
    return np.linalg.norm(p1 - p2)

# ================================
# Set Screen Dimensions
# ================================
WIDTH, HEIGHT = 1200, 900

# ================================
# Parse the KML File
# ================================
def parse_kml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    namespace = {'kml': 'http://www.opengis.net/kml/2.2'}

    taxiways = []
    gates = []

    for placemark in root.findall('.//kml:Placemark', namespace):
        # Extract LineStrings (taxiways and runways)
        line_string = placemark.find('.//kml:LineString/kml:coordinates', namespace)
        if line_string is not None:
            line_coords = line_string.text.strip().split()
            path = []
            for coord in line_coords:
                lon, lat, _ = map(float, coord.split(','))
                path.append((lon, lat))
            taxiway = {'path': path}
            name = placemark.find('kml:name', namespace)
            if name is not None:
                placemark_name = name.text.strip()
                taxiway['name'] = placemark_name
                match = re.search(r'\b(L1|L2|L3)\b', placemark_name)
                if match:
                    taxiway['code'] = match.group(1)
                else:
                    taxiway['code'] = ''
                if 'CENTRELINE' in placemark_name.upper():
                    taxiway['is_runway'] = True
            taxiways.append(taxiway)

        # Extract Points (gates)
        point = placemark.find('.//kml:Point/kml:coordinates', namespace)
        if point is not None:
            coord = point.text.strip().split(',')
            lon, lat, _ = map(float, coord)
            gates.append({'position': (lon, lat)})

    return taxiways, gates

# ================================
# Build the NetworkX Graph
# ================================
def build_graph(taxiways, gates, width, height):
    all_taxiway_coords = [coord for taxiway in taxiways for coord in taxiway['path']]
    all_gate_coords = [gate['position'] for gate in gates]
    all_coords = all_taxiway_coords + all_gate_coords

    lon_min = min(coord[0] for coord in all_coords)
    lon_max = max(coord[0] for coord in all_coords)
    lat_min = min(coord[1] for coord in all_coords)
    lat_max = max(coord[1] for coord in all_coords)

    margin = 0.05
    lon_range = lon_max - lon_min
    lat_range = lat_max - lat_min
    lon_min -= lon_range * margin
    lon_max += lon_range * margin
    lat_min -= lat_range * margin
    lat_max += lat_range * margin

    def convert_coords(lon, lat):
        x = (lon - lon_min) / (lon_max - lon_min) * width
        y = height - (lat - lat_min) / (lat_max - lat_min) * height
        return np.array([x, y])

    for taxiway in taxiways:
        pixel_path = [convert_coords(lon, lat) for lon, lat in taxiway['path']]
        taxiway['pixel_path'] = pixel_path

    for gate in gates:
        gate['pixel_position'] = convert_coords(*gate['position'])

    G = nx.Graph()
    node_id_counter = 0
    node_positions = {}
    for idx, taxiway in enumerate(taxiways):
        path = taxiway['pixel_path']
        taxiway_name = taxiway.get('name', '')
        taxiway_code = taxiway.get('code', '')
        taxiway_nodes = []
        for i, point in enumerate(path):
            node_id = node_id_counter
            node_id_counter += 1
            G.add_node(node_id, pos=point)
            G.nodes[node_id]['taxiway_name'] = taxiway_name
            G.nodes[node_id]['taxiway_code'] = taxiway_code
            if taxiway.get('is_runway'):
                G.nodes[node_id]['is_runway_node'] = True
            node_positions[node_id] = point
            taxiway_nodes.append(node_id)
            if i > 0:
                prev_node_id = taxiway_nodes[i - 1]
                G.add_edge(prev_node_id, node_id, weight=distance(point, node_positions[prev_node_id]))
        taxiway['node_ids'] = taxiway_nodes

    def connect_taxiway_endpoints(G, taxiways):
        CONNECTION_THRESHOLD = 30
        for taxiway1 in taxiways:
            for taxiway2 in taxiways:
                if taxiway1 == taxiway2:
                    continue
                endpoints1 = [taxiway1['node_ids'][0], taxiway1['node_ids'][-1]]
                endpoints2 = [taxiway2['node_ids'][0], taxiway2['node_ids'][-1]]
                for node1 in endpoints1:
                    for node2 in endpoints2:
                        pos1 = G.nodes[node1]['pos']
                        pos2 = G.nodes[node2]['pos']
                        if np.linalg.norm(pos1 - pos2) < CONNECTION_THRESHOLD:
                            if not G.has_edge(node1, node2):
                                G.add_edge(node1, node2, weight=np.linalg.norm(pos1 - pos2))

    def connect_runways_to_taxiways(G, taxiways):
        CONNECTION_THRESHOLD = 30
        for runway in [t for t in taxiways if t.get('is_runway')]:
            for taxiway in [t for t in taxiways if not t.get('is_runway')]:
                for runway_node in runway['node_ids']:
                    for taxiway_node in taxiway['node_ids']:
                        pos_runway = G.nodes[runway_node]['pos']
                        pos_taxiway = G.nodes[taxiway_node]['pos']
                        if np.linalg.norm(pos_runway - pos_taxiway) < CONNECTION_THRESHOLD:
                            if not G.has_edge(runway_node, taxiway_node):
                                G.add_edge(runway_node, taxiway_node, weight=np.linalg.norm(pos_runway - pos_taxiway))

    connect_taxiway_endpoints(G, taxiways)
    connect_runways_to_taxiways(G, taxiways)

    # Ensure all edges have a weight of 1
    for edge in G.edges():
        G[edge[0]][edge[1]]['weight'] = 1

    return G, node_positions, convert_coords

# ================================
# Add Predefined Gates and Runway Endpoints
# ================================
def add_predefined_nodes(G, node_positions, gate_coordinates, runway_endpoints, convert_coords, node_id_counter):
    for gate_coord in gate_coordinates:
        gate_pixel_position = convert_coords(*gate_coord)
        node_id = node_id_counter
        node_id_counter += 1
        G.add_node(node_id, pos=gate_pixel_position, is_gate=True)
        node_positions[node_id] = gate_pixel_position

    for runway_coord in runway_endpoints:
        runway_pixel_position = convert_coords(*runway_coord)
        node_id = node_id_counter
        node_id_counter += 1
        G.add_node(node_id, pos=runway_pixel_position, is_runway_endpoint=True)
        node_positions[node_id] = runway_pixel_position

    return G, node_positions, node_id_counter

# ================================
# Custom Multi-Agent Environment
# ================================
class RoadIntersectionEnv(MultiAgentEnv):
    def __init__(self, config, callback=None):
        super().__init__()
        self.graph = config["graph"]
        self.total_agents = config["num_agents"]
        self.agents = [f"car_{i}" for i in range(self.total_agents)]
        self.pos = {agent: None for agent in self.agents}
        self.distance_travelled = {agent: 0 for agent in self.agents}
        self.callback = callback

        # Use Discrete spaces to represent each agent's current node in the graph
        self.observation_space = spaces.Discrete(len(self.graph.nodes))
        self.action_space = spaces.Discrete(len(self.graph.nodes))

    def reset(self, seed=None, options=None):
        if seed is not None:
            np.random.seed(seed)

        # Randomly assign starting positions from gates and runway endpoints
        available_nodes = [node for node, data in self.graph.nodes(data=True) if data.get('is_gate') or data.get('is_runway_endpoint')]
        for agent in self.agents:
            self.pos[agent] = np.random.choice(available_nodes)
            self.distance_travelled[agent] = 0
        self.done_agents = set()
        obs = {agent: self.pos[agent] for agent in self.agents}  # Return observations as a dictionary

        # Update callback with current positions
        if self.callback:
            self.callback.update_positions(self.graph, {agent: self.graph.nodes[self.pos[agent]]['pos'] for agent in self.agents})

        return obs, {}

    def step(self, action_dict):
        rewards = {agent: 0 for agent in self.agents}
        dones = {agent: False for agent in self.agents}
        infos = {}
        truncateds = {agent: False for agent in self.agents}

        # Move each agent
        for agent, action in action_dict.items():
            if agent in self.done_agents:
                continue

            current_node = self.pos[agent]
            next_node = action

            # Ensure next_node is a valid neighbor of the current node
            if current_node in self.graph and next_node in self.graph[current_node]:
                self.pos[agent] = next_node
                self.distance_travelled[agent] += self.graph[current_node][next_node]['weight']
                rewards[agent] = -0.1  # Small negative reward to encourage shorter paths
            else:
                # Invalid action, penalize the agent
                rewards[agent] -= 1

        # Detect collisions
        occupied_nodes = {}
        for agent in self.agents:
            if agent in self.done_agents:
                continue

            pos = self.pos[agent]
            if pos not in occupied_nodes:
                occupied_nodes[pos] = agent
            else:
                # Collision occurred
                rewards[agent] -= 5  # Penalty for collision
                other_agent = occupied_nodes[pos]
                rewards[other_agent] -= 5
                dones[agent] = True
                dones[other_agent] = True
                self.done_agents.add(agent)
                self.done_agents.add(other_agent)

        # Update done flags for all agents
        for agent in self.agents:
            if agent not in self.done_agents:
                dones[agent] = False
            else:
                dones[agent] = True
                truncateds[agent] = True  # Mark as truncated to properly handle episode end

        # Check if all agents are done
        all_done = len(self.done_agents) == self.total_agents
        dones['__all__'] = all_done
        truncateds['__all__'] = all_done

        obs = {agent: self.pos[agent] for agent in self.agents if agent not in self.done_agents}  # Return observations as a dictionary
        infos = {agent: {} for agent in self.agents if agent not in self.done_agents}  # Update infos to match the current observation keys

        # Update callback with current positions
        if self.callback:
            self.callback.update_positions(self.graph, {agent: self.graph.nodes[self.pos[agent]]['pos'] for agent in self.agents})

        # Render the environment to visualize agent movement
        self.render()

        return obs, rewards, dones, truncateds, infos

    def render(self):
        if self.callback and self.callback.graph and self.callback.current_positions:
            self.callback.fig.suptitle("Agent Movement Visualization", fontsize=16)
            self.callback.ax2.clear()
            pos = nx.get_node_attributes(self.graph, 'pos')
            nx.draw(self.graph, pos, node_color='lightblue', edge_color='gray', with_labels=False, node_size=20)
            agent_positions = np.array(list(self.pos.values()))
            agent_coordinates = [self.graph.nodes[node]['pos'] for node in agent_positions if node is not None]
            if agent_coordinates:
                agent_coordinates = np.array(agent_coordinates)
                plt.scatter(agent_coordinates[:, 0], agent_coordinates[:, 1], color='red', s=100, label="Agents")
            plt.title("Agent Movement in Environment")
            plt.legend()
            plt.pause(0.01)

# ================================
# Main Function
# ================================
def main():
    # Include global variables required by the environment
    global taxiways

    # ================================
    # Parse KML and Build Graph
    # ================================
    file_path = "reduced_nodes.kml"  # Update this path to your KML file
    taxiways, gates = parse_kml(file_path)
    G, node_positions, convert_coords = build_graph(taxiways, gates, WIDTH, HEIGHT)

    # Predefined gates and runway endpoints for agents to start and end
    gate_coordinates = [
        (77.71046165300096, 13.20355775032989),
        (77.71066072998956, 13.20166801620613),
        (77.70315360361252, 13.20165318995302),
        (77.71305689010458, 13.201637816132),
        (77.70195813939766, 13.20169406641297),
        (77.71149825860202, 13.20165032306983),
        (77.72279066770344, 13.19968338101175),
        (77.7132055646255, 13.20167917723071),
        (77.7089924860226, 13.20357466411958),
        (77.71648635375834, 13.20336974504588)
    ]

    runway_endpoints = [
        (77.68686735208298, 13.20718308006773),
        (77.7222951686759, 13.20687066472886),
        (77.69060216489893, 13.18975920793089),
        (77.72642499411846, 13.18944676842937)
    ]

    # Add predefined gates and runway endpoints to the graph
    node_id_counter = len(G.nodes())
    G, node_positions, node_id_counter = add_predefined_nodes(
        G, node_positions, gate_coordinates, runway_endpoints,
        convert_coords=convert_coords,
        node_id_counter=node_id_counter
    )

    # ================================
    # Initialize Ray
    # ================================
    ray.init(ignore_reinit_error=True, log_to_driver=True)

    # ================================
    # Register the Environment
    # ================================
    from ray.tune.registry import register_env

    callback = RealTimePlottingCallback()

    def env_creator(env_config):
        return RoadIntersectionEnv(env_config, callback=callback)

    register_env("RoadIntersectionEnv", lambda env_config: env_creator(env_config))

    # ================================
    # Define Policy Mapping Function
    # ================================
    def policy_mapping_fn(agent_id, *args, **kwargs):
        return "shared_policy"

    # ================================
    # RLlib Training Configuration
    # ================================
    temp_env = RoadIntersectionEnv({"graph": G, "num_agents": 3}, callback=callback)

    config = {
        "env": "RoadIntersectionEnv",
        "env_config": {
            "graph": G,
            "num_agents": 3
        },
        "framework": "torch",
        "num_workers": 1,  # Run in the main thread for rendering
        "num_envs_per_worker": 1,
        "num_gpus": 1,  # Set to 1 if you have a GPU and want to use it
        "train_batch_size": 4000,
        "sgd_minibatch_size": 128,
        "lr": 1e-3,
        "gamma": 0.99,
        "log_level": "WARN",
        "multiagent": {
            "policies": {
                "shared_policy": (None, temp_env.observation_space, temp_env.action_space, {})
            },
            "policy_mapping_fn": policy_mapping_fn,
            "policies_to_train": ["shared_policy"],
        },
    }

    # Training using tune.run with Custom Callback for Real-Time Visualization
    tune.run(
        "PPO",
        config=config,
        stop={"training_iteration": 25},
        storage_path="D:/aniru-windows/Capstone-Implementation/Localrun_ray_checkpoints",
        checkpoint_freq=5,
        checkpoint_at_end=True,
        verbose=1,
        callbacks=[callback]
    )

    # Clean up
    ray.shutdown()

# ================================
# Run the Main Function
# ================================
if __name__ == "__main__":
    main()

2024-11-29 23:37:04,305	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_envs_per_worker` has been deprecated. Use `AlgorithmConfig.num_envs_per_env_runner` instead. This will raise an error in the future!
(PPO pid=40028) 2024-11-29 23:37:18,085	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_envs_per_worker` has been deprecated. Use `AlgorithmConfig.num_envs_per_env_runner` instead. This will raise an error in the future!
(PPO pid=40028) Trainable.setup took 20.531 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(PPO pid=40028) Install gputil for GPU system monitoring.
2024-11-29 23:37:40,111	ERROR tune_controller.py:1331 -- Trial task failed for trial PPO_RoadIntersectionEnv_bd0fb_00000
Traceback (most recent call last):
  File "C:\Users\aniru\miniconda3\envs\env1\Lib\site-packages\ray\air\execution\_internal\event_manager.py", line 110, in resolve_future
    result = r

TuneError: ('Trials did not complete', [PPO_RoadIntersectionEnv_bd0fb_00000])

In [ ]:
# D:/aniru-windows/Capstone-Implementation/Localrun_ray_checkpoints
# 
#

# Matplotlib but no ray tune

In [1]:
import numpy as np
import networkx as nx
import gymnasium as gym
from gymnasium import spaces
import ray
from ray import tune
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from ray.tune import Callback
import sys
import os
import xml.etree.ElementTree as ET
import re
import matplotlib
import matplotlib.pyplot as plt
import time

# Set matplotlib backend to ensure compatibility with interactive plots
matplotlib.use('TkAgg')

# Custom Callback for Real-Time Plotting
class RealTimePlottingCallback(Callback):
    def __init__(self):
        self.iterations = []
        self.mean_rewards = []
        self.fig, self.ax1 = plt.subplots(figsize=(8, 6))
        plt.ion()  # Interactive mode for real-time plotting
        plt.show()
        self.current_positions = None
        self.graph = None

    def on_trial_result(self, iteration, trials, trial, result, **info):
        mean_reward = result.get("episode_reward_mean", 0)
        self.iterations.append(iteration)
        self.mean_rewards.append(mean_reward)

        # Update the reward plot
        self.ax1.clear()
        self.ax1.plot(self.iterations, self.mean_rewards, label="Mean Episode Reward")
        self.ax1.set_xlabel("Training Iteration")
        self.ax1.set_ylabel("Mean Episode Reward")
        self.ax1.set_title("Training Progress")
        self.ax1.legend()
        self.ax1.grid()

        plt.pause(0.01)  # Pause to allow the plot to update

    def update_positions(self, graph, current_positions):
        self.graph = graph
        self.current_positions = current_positions

        # Update the agent movement plot
        plt.figure("Agent Movement Visualization")
        plt.clf()
        pos = nx.get_node_attributes(self.graph, 'pos')
        nx.draw(self.graph, pos, node_color='lightblue', edge_color='gray', with_labels=False, node_size=20)
        agent_positions = np.array(list(self.current_positions.values()))
        plt.scatter(agent_positions[:, 0], agent_positions[:, 1], color='red', s=100, label="Agents")
        plt.title("Agent Movement in Environment")
        plt.legend()
        plt.pause(0.01)

# ================================
# Define the Distance Function at the Module Level
# ================================
def distance(p1, p2):
    return np.linalg.norm(p1 - p2)

# ================================
# Set Screen Dimensions
# ================================
WIDTH, HEIGHT = 1200, 900

# ================================
# Parse the KML File
# ================================
def parse_kml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    namespace = {'kml': 'http://www.opengis.net/kml/2.2'}

    taxiways = []
    gates = []

    for placemark in root.findall('.//kml:Placemark', namespace):
        # Extract LineStrings (taxiways and runways)
        line_string = placemark.find('.//kml:LineString/kml:coordinates', namespace)
        if line_string is not None:
            line_coords = line_string.text.strip().split()
            path = []
            for coord in line_coords:
                lon, lat, _ = map(float, coord.split(','))
                path.append((lon, lat))
            taxiway = {'path': path}
            name = placemark.find('kml:name', namespace)
            if name is not None:
                placemark_name = name.text.strip()
                taxiway['name'] = placemark_name
                match = re.search(r'\b(L1|L2|L3)\b', placemark_name)
                if match:
                    taxiway['code'] = match.group(1)
                else:
                    taxiway['code'] = ''
                if 'CENTRELINE' in placemark_name.upper():
                    taxiway['is_runway'] = True
            taxiways.append(taxiway)

        # Extract Points (gates)
        point = placemark.find('.//kml:Point/kml:coordinates', namespace)
        if point is not None:
            coord = point.text.strip().split(',')
            lon, lat, _ = map(float, coord)
            gates.append({'position': (lon, lat)})

    return taxiways, gates

# ================================
# Build the NetworkX Graph
# ================================
def build_graph(taxiways, gates, width, height):
    all_taxiway_coords = [coord for taxiway in taxiways for coord in taxiway['path']]
    all_gate_coords = [gate['position'] for gate in gates]
    all_coords = all_taxiway_coords + all_gate_coords

    lon_min = min(coord[0] for coord in all_coords)
    lon_max = max(coord[0] for coord in all_coords)
    lat_min = min(coord[1] for coord in all_coords)
    lat_max = max(coord[1] for coord in all_coords)

    margin = 0.05
    lon_range = lon_max - lon_min
    lat_range = lat_max - lat_min
    lon_min -= lon_range * margin
    lon_max += lon_range * margin
    lat_min -= lat_range * margin
    lat_max += lat_range * margin

    def convert_coords(lon, lat):
        x = (lon - lon_min) / (lon_max - lon_min) * width
        y = height - (lat - lat_min) / (lat_max - lat_min) * height
        return np.array([x, y])

    for taxiway in taxiways:
        pixel_path = [convert_coords(lon, lat) for lon, lat in taxiway['path']]
        taxiway['pixel_path'] = pixel_path

    for gate in gates:
        gate['pixel_position'] = convert_coords(*gate['position'])

    G = nx.Graph()
    node_id_counter = 0
    node_positions = {}
    for idx, taxiway in enumerate(taxiways):
        path = taxiway['pixel_path']
        taxiway_name = taxiway.get('name', '')
        taxiway_code = taxiway.get('code', '')
        taxiway_nodes = []
        for i, point in enumerate(path):
            node_id = node_id_counter
            node_id_counter += 1
            G.add_node(node_id, pos=point)
            G.nodes[node_id]['taxiway_name'] = taxiway_name
            G.nodes[node_id]['taxiway_code'] = taxiway_code
            if taxiway.get('is_runway'):
                G.nodes[node_id]['is_runway_node'] = True
            node_positions[node_id] = point
            taxiway_nodes.append(node_id)
            if i > 0:
                prev_node_id = taxiway_nodes[i - 1]
                G.add_edge(prev_node_id, node_id, weight=distance(point, node_positions[prev_node_id]))
        taxiway['node_ids'] = taxiway_nodes

    def connect_taxiway_endpoints(G, taxiways):
        CONNECTION_THRESHOLD = 30
        for taxiway1 in taxiways:
            for taxiway2 in taxiways:
                if taxiway1 == taxiway2:
                    continue
                endpoints1 = [taxiway1['node_ids'][0], taxiway1['node_ids'][-1]]
                endpoints2 = [taxiway2['node_ids'][0], taxiway2['node_ids'][-1]]
                for node1 in endpoints1:
                    for node2 in endpoints2:
                        pos1 = G.nodes[node1]['pos']
                        pos2 = G.nodes[node2]['pos']
                        if np.linalg.norm(pos1 - pos2) < CONNECTION_THRESHOLD:
                            if not G.has_edge(node1, node2):
                                G.add_edge(node1, node2, weight=np.linalg.norm(pos1 - pos2))

    def connect_runways_to_taxiways(G, taxiways):
        CONNECTION_THRESHOLD = 30
        for runway in [t for t in taxiways if t.get('is_runway')]:
            for taxiway in [t for t in taxiways if not t.get('is_runway')]:
                for runway_node in runway['node_ids']:
                    for taxiway_node in taxiway['node_ids']:
                        pos_runway = G.nodes[runway_node]['pos']
                        pos_taxiway = G.nodes[taxiway_node]['pos']
                        if np.linalg.norm(pos_runway - pos_taxiway) < CONNECTION_THRESHOLD:
                            if not G.has_edge(runway_node, taxiway_node):
                                G.add_edge(runway_node, taxiway_node, weight=np.linalg.norm(pos_runway - pos_taxiway))

    connect_taxiway_endpoints(G, taxiways)
    connect_runways_to_taxiways(G, taxiways)

    # Ensure all edges have a weight of 1
    for edge in G.edges():
        G[edge[0]][edge[1]]['weight'] = 1

    return G, node_positions, convert_coords

# ================================
# Add Predefined Gates and Runway Endpoints
# ================================
def add_predefined_nodes(G, node_positions, gate_coordinates, runway_endpoints, convert_coords, node_id_counter):
    for gate_coord in gate_coordinates:
        gate_pixel_position = convert_coords(*gate_coord)
        node_id = node_id_counter
        node_id_counter += 1
        G.add_node(node_id, pos=gate_pixel_position, is_gate=True)
        node_positions[node_id] = gate_pixel_position

    for runway_coord in runway_endpoints:
        runway_pixel_position = convert_coords(*runway_coord)
        node_id = node_id_counter
        node_id_counter += 1
        G.add_node(node_id, pos=runway_pixel_position, is_runway_endpoint=True)
        node_positions[node_id] = runway_pixel_position

    return G, node_positions, node_id_counter

# ================================
# Custom Multi-Agent Environment
# ================================
class RoadIntersectionEnv(MultiAgentEnv):
    def __init__(self, config, callback=None):
        super().__init__()
        self.graph = config["graph"]
        self.total_agents = config["num_agents"]
        self.agents = [f"car_{i}" for i in range(self.total_agents)]
        self.pos = {agent: None for agent in self.agents}
        self.distance_travelled = {agent: 0 for agent in self.agents}
        self.callback = callback

        # Use Discrete spaces to represent each agent's current node in the graph
        self.observation_space = spaces.Discrete(len(self.graph.nodes))
        self.action_space = spaces.Discrete(len(self.graph.nodes))

    def reset(self, seed=None, options=None):
        if seed is not None:
            np.random.seed(seed)

        # Randomly assign starting positions from gates and runway endpoints
        available_nodes = [node for node, data in self.graph.nodes(data=True) if data.get('is_gate') or data.get('is_runway_endpoint')]
        for agent in self.agents:
            self.pos[agent] = np.random.choice(available_nodes)
            self.distance_travelled[agent] = 0
        self.done_agents = set()
        obs = {agent: self.pos[agent] for agent in self.agents}  # Return observations as a dictionary

        # Update callback with current positions
        if self.callback:
            self.callback.update_positions(self.graph, {agent: self.graph.nodes[self.pos[agent]]['pos'] for agent in self.agents})

        return obs, {}

    def step(self, action_dict):
        rewards = {agent: 0 for agent in self.agents}
        dones = {agent: False for agent in self.agents}
        infos = {}
        truncateds = {agent: False for agent in self.agents}

        # Move each agent
        for agent, action in action_dict.items():
            if agent in self.done_agents:
                continue

            current_node = self.pos[agent]
            next_node = action

            # Ensure next_node is a valid neighbor of the current node
            if current_node in self.graph and next_node in self.graph[current_node]:
                self.pos[agent] = next_node
                self.distance_travelled[agent] += self.graph[current_node][next_node]['weight']
                rewards[agent] = -0.1  # Small negative reward to encourage shorter paths
            else:
                # Invalid action, penalize the agent
                rewards[agent] -= 1

        # Detect collisions
        occupied_nodes = {}
        for agent in self.agents:
            if agent in self.done_agents:
                continue

            pos = self.pos[agent]
            if pos not in occupied_nodes:
                occupied_nodes[pos] = agent
            else:
                # Collision occurred
                rewards[agent] -= 5  # Penalty for collision
                other_agent = occupied_nodes[pos]
                rewards[other_agent] -= 5
                dones[agent] = True
                dones[other_agent] = True
                self.done_agents.add(agent)
                self.done_agents.add(other_agent)

        # Update done flags for all agents
        for agent in self.agents:
            if agent not in self.done_agents:
                dones[agent] = False
            else:
                dones[agent] = True
                truncateds[agent] = True  # Mark as truncated to properly handle episode end

        # Check if all agents are done
        all_done = len(self.done_agents) == self.total_agents
        dones['__all__'] = all_done
        truncateds['__all__'] = all_done

        obs = {agent: self.pos[agent] for agent in self.agents if agent not in self.done_agents}  # Return observations as a dictionary
        infos = {agent: {} for agent in self.agents if agent not in self.done_agents}  # Update infos to match the current observation keys

        # Update callback with current positions
        if self.callback:
            self.callback.update_positions(self.graph, {agent: self.graph.nodes[self.pos[agent]]['pos'] for agent in self.agents})

        return obs, rewards, dones, truncateds, infos

# ================================
# Main Function
# ================================
def main():
    # Include global variables required by the environment
    global taxiways

    # ================================
    # Parse KML and Build Graph
    # ================================
    file_path = "reduced_nodes.kml"  # Update this path to your KML file
    taxiways, gates = parse_kml(file_path)
    G, node_positions, convert_coords = build_graph(taxiways, gates, WIDTH, HEIGHT)

    # Predefined gates and runway endpoints for agents to start and end
    gate_coordinates = [
        (77.71046165300096, 13.20355775032989),
        (77.71066072998956, 13.20166801620613),
        (77.70315360361252, 13.20165318995302),
        (77.71305689010458, 13.201637816132),
        (77.70195813939766, 13.20169406641297),
        (77.71149825860202, 13.20165032306983),
        (77.72279066770344, 13.19968338101175),
        (77.7132055646255, 13.20167917723071),
        (77.7089924860226, 13.20357466411958),
        (77.71648635375834, 13.20336974504588)
    ]

    runway_endpoints = [
        (77.68686735208298, 13.20718308006773),
        (77.7222951686759, 13.20687066472886),
        (77.69060216489893, 13.18975920793089),
        (77.72642499411846, 13.18944676842937)
    ]

    # Add predefined gates and runway endpoints to the graph
    node_id_counter = len(G.nodes())
    G, node_positions, node_id_counter = add_predefined_nodes(
        G, node_positions, gate_coordinates, runway_endpoints,
        convert_coords=convert_coords,
        node_id_counter=node_id_counter
    )

    # ================================
    # Initialize Ray
    # ================================
    ray.init(ignore_reinit_error=True, log_to_driver=True)

    # ================================
    # Register the Environment
    # ================================
    from ray.tune.registry import register_env

    callback = RealTimePlottingCallback()

    def env_creator(env_config):
        return RoadIntersectionEnv(env_config, callback=callback)

    register_env("RoadIntersectionEnv", lambda env_config: env_creator(env_config))

    # ================================
    # Define Policy Mapping Function
    # ================================
    def policy_mapping_fn(agent_id, *args, **kwargs):
        return "shared_policy"

    # ================================
    # RLlib Training Configuration
    # ================================
    temp_env = RoadIntersectionEnv({"graph": G, "num_agents": 3}, callback=callback)

    config = {
        "env": "RoadIntersectionEnv",
        "env_config": {
            "graph": G,
            "num_agents": 3
        },
        "framework": "torch",
        "num_workers": 1,  # Run in the main thread for rendering
        "num_envs_per_worker": 1,
        "num_gpus": 1,  # Set to 1 if you have a GPU and want to use it
        "train_batch_size": 4000,
        "sgd_minibatch_size": 128,
        "lr": 1e-3,
        "gamma": 0.99,
        "log_level": "WARN",
        "multiagent": {
            "policies": {
                "shared_policy": (None, temp_env.observation_space, temp_env.action_space, {})
            },
            "policy_mapping_fn": policy_mapping_fn,
            "policies_to_train": ["shared_policy"],
        },
    }

    # Training using tune.run with Custom Callback for Real-Time Visualization
    tune.run(
        "PPO",
        config=config,
        stop={"training_iteration": 50},
        storage_path="D:/aniru-windows/Capstone-Implementation/Localrun_ray_checkpoints",
        checkpoint_freq=5,
        checkpoint_at_end=True,
        verbose=1,
        callbacks=[callback]
    )

    # Clean up
    ray.shutdown()

# ================================
# Run the Main Function
# ================================
if __name__ == "__main__":
    main()

2024-12-01 11:05:16,802	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_envs_per_worker` has been deprecated. Use `AlgorithmConfig.num_envs_per_env_runner` instead. This will raise an error in the future!
(PPO pid=29284) 2024-12-01 11:05:29,839	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_envs_per_worker` has been deprecated. Use `AlgorithmConfig.num_envs_per_env_runner` instead. This will raise an error in the future!
(PPO pid=29284) Trainable.setup took 22.062 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(PPO pid=29284) Install gputil for GPU system monitoring.


(RolloutWorker pid=25588) Figure(800x600)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(6

(PPO pid=29284) 2024-12-01 11:06:54,312	WARNING rollout_ops.py:115 -- No samples returned from remote workers. If you have a slow environment or model, consider increasing the `sample_timeout_s` or decreasing the `rollout_fragment_length` in `AlgorithmConfig.env_runners().


(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(6

(PPO pid=29284) 2024-12-01 11:08:56,275	WARNING rollout_ops.py:115 -- No samples returned from remote workers. If you have a slow environment or model, consider increasing the `sample_timeout_s` or decreasing the `rollout_fragment_length` in `AlgorithmConfig.env_runners().


...

(PPO pid=29284) 2024-12-01 11:37:21,053	WARNING rollout_ops.py:115 -- No samples returned from remote workers. If you have a slow environment or model, consider increasing the `sample_timeout_s` or decreasing the `rollout_fragment_length` in `AlgorithmConfig.env_runners().


(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(6

2024-12-01 11:38:17,998	WARNING tune.py:219 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2024-12-01 11:38:18,048	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to 'D:/aniru-windows/Capstone-Implementation/Localrun_ray_checkpoints/PPO_2024-12-01_11-05-16' in 0.0478s.


(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(640x480)
(RolloutWorker pid=25588) Figure(6

2024-12-01 11:38:28,187	INFO tune.py:1041 -- Total run time: 1991.66 seconds (1981.20 seconds for the tuning loop).
2024-12-01 11:38:28,189	WARNING tune.py:1056 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: tune.run(..., resume=True)


In [ ]:
# D:/aniru-windows/Capstone-Implementation/Localrun_ray_checkpoints

# Visualising only checkpoints without training 

In [9]:
import numpy as np
import networkx as nx
import gymnasium as gym
from gymnasium import spaces
import ray
from ray import tune
from ray.rllib.algorithms.ppo import PPOConfig, PPO
from ray.rllib.env.multi_agent_env import MultiAgentEnv
import sys
import os
import xml.etree.ElementTree as ET
import re
import pygame

# ================================
# Define the Distance Function at the Module Level
# ================================
def distance(p1, p2):
    return np.linalg.norm(p1 - p2)

# ================================
# Set Screen Dimensions
# ================================
WIDTH, HEIGHT = 1200, 900

# ================================
# Parse the KML File
# ================================
def parse_kml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    namespace = {'kml': 'http://www.opengis.net/kml/2.2'}

    taxiways = []
    gates = []

    for placemark in root.findall('.//kml:Placemark', namespace):
        # Extract LineStrings (taxiways and runways)
        line_string = placemark.find('.//kml:LineString/kml:coordinates', namespace)
        if line_string is not None:
            line_coords = line_string.text.strip().split()
            path = []
            for coord in line_coords:
                lon, lat, _ = map(float, coord.split(','))
                path.append((lon, lat))
            taxiway = {'path': path}
            name = placemark.find('kml:name', namespace)
            if name is not None:
                placemark_name = name.text.strip()
                taxiway['name'] = placemark_name
            taxiways.append(taxiway)

        # Extract Points (gates)
        point = placemark.find('.//kml:Point/kml:coordinates', namespace)
        if point is not None:
            coord = point.text.strip().split(',')
            lon, lat, _ = map(float, coord)
            gates.append({'position': (lon, lat)})

    return taxiways, gates

# ================================
# Build the NetworkX Graph
# ================================
def build_graph(taxiways, gates, width, height):
    all_taxiway_coords = [coord for taxiway in taxiways for coord in taxiway['path']]
    all_gate_coords = [gate['position'] for gate in gates]
    all_coords = all_taxiway_coords + all_gate_coords

    lon_min = min(coord[0] for coord in all_coords)
    lon_max = max(coord[0] for coord in all_coords)
    lat_min = min(coord[1] for coord in all_coords)
    lat_max = max(coord[1] for coord in all_coords)

    margin = 0.05
    lon_range = lon_max - lon_min
    lat_range = lat_max - lat_min
    lon_min -= lon_range * margin
    lon_max += lon_range * margin
    lat_min -= lat_range * margin
    lat_max += lat_range * margin

    def convert_coords(lon, lat):
        x = (lon - lon_min) / (lon_max - lon_min) * width
        y = height - (lat - lat_min) / (lat_max - lat_min) * height
        return np.array([x, y])

    for taxiway in taxiways:
        pixel_path = [convert_coords(lon, lat) for lon, lat in taxiway['path']]
        taxiway['pixel_path'] = pixel_path

    for gate in gates:
        gate['pixel_position'] = convert_coords(*gate['position'])

    G = nx.Graph()
    node_id_counter = 0
    node_positions = {}
    for idx, taxiway in enumerate(taxiways):
        path = taxiway['pixel_path']
        taxiway_name = taxiway.get('name', '')
        taxiway_nodes = []
        for i, point in enumerate(path):
            node_id = node_id_counter
            node_id_counter += 1
            G.add_node(node_id, pos=point)
            G.nodes[node_id]['taxiway_name'] = taxiway_name
            node_positions[node_id] = point
            taxiway_nodes.append(node_id)
            if i > 0:
                prev_node_id = taxiway_nodes[i - 1]
                G.add_edge(prev_node_id, node_id, weight=distance(point, node_positions[prev_node_id]))
        taxiway['node_ids'] = taxiway_nodes

    # Ensure all edges have a weight of 1
    for edge in G.edges():
        G[edge[0]][edge[1]]['weight'] = 1

    return G, node_positions, convert_coords

# ================================
# Custom Multi-Agent Environment
# ================================
class RoadIntersectionEnv(MultiAgentEnv):
    def __init__(self, config, callback=None):
        super().__init__()
        self.graph = config["graph"]
        self.total_agents = config["num_agents"]
        self.agents = [f"car_{i}" for i in range(self.total_agents)]
        self.pos = {agent: None for agent in self.agents}
        self.distance_travelled = {agent: 0 for agent in self.agents}
        self.destinations = {agent: None for agent in self.agents}
        self.callback = callback

        self.width, self.height = WIDTH, HEIGHT
        self.screen = None

        # Define observation and action spaces
        self.observation_space = spaces.Discrete(len(self.graph.nodes))
        self.action_space = spaces.Discrete(len(self.graph.nodes))

        # Predefined gates and runway endpoints for agents to start and end
        self.gate_coordinates = [
            (77.71046165300096, 13.20355775032989),
            (77.71066072998956, 13.20166801620613),
            (77.70315360361252, 13.20165318995302),
            (77.71305689010458, 13.201637816132),
            (77.70195813939766, 13.20169406641297),
            (77.71149825860202, 13.20165032306983),
            (77.72279066770344, 13.19968338101175),
            (77.7132055646255, 13.20167917723071),
            (77.7089924860226, 13.20357466411958),
            (77.71648635375834, 13.20336974504588)
        ]

        self.runway_endpoints = [
            (77.68686735208298, 13.20718308006773),
            (77.7222951686759, 13.20687066472886),
            (77.69060216489893, 13.18975920793089),
            (77.72642499411846, 13.18944676842937)
        ]

        # Pygame initialization
        pygame.init()
        self.screen = pygame.display.set_mode((self.width, self.height))
        pygame.display.set_caption("Airport Taxiway Management")
        self.clock = pygame.time.Clock()

    def reset(self, seed=None, options=None):
        if seed is not None:
            np.random.seed(seed)

        # Flatten the list of coordinates for use in np.random.choice
        all_coordinates = self.gate_coordinates + self.runway_endpoints

        for agent in self.agents:
            start_coord = all_coordinates[np.random.choice(len(all_coordinates))]
            dest_coord = all_coordinates[np.random.choice(len(all_coordinates))]
            while dest_coord == start_coord:
                dest_coord = all_coordinates[np.random.choice(len(all_coordinates))]

            # Convert start and destination coordinates to node positions
            start_pos = self._convert_coords(*start_coord)
            dest_pos = self._convert_coords(*dest_coord)

            # Find the closest node to the start and destination positions
            start_node = self._find_closest_node(start_pos)
            dest_node = self._find_closest_node(dest_pos)

            self.pos[agent] = start_node
            self.destinations[agent] = dest_node
            self.distance_travelled[agent] = 0

        self.done_agents = set()
        obs = {agent: self.pos[agent] for agent in self.agents}

        if self.callback:
            self.callback.update_positions(self.graph, {agent: self.graph.nodes[self.pos[agent]]['pos'] for agent in self.agents})

        # Render the environment state after reset
        self.render()

        return obs, {}

    def _convert_coords(self, lon, lat):
        lon_min, lon_max, lat_min, lat_max = self._get_bounds()
        x = (lon - lon_min) / (lon_max - lon_min) * self.width
        y = self.height - (lat - lat_min) / (lat_max - lat_min) * self.height
        return np.array([x, y])

    def _get_bounds(self):
        all_coords = self.gate_coordinates + self.runway_endpoints
        lon_min = min(coord[0] for coord in all_coords)
        lon_max = max(coord[0] for coord in all_coords)
        lat_min = min(coord[1] for coord in all_coords)
        lat_max = max(coord[1] for coord in all_coords)
        return lon_min, lon_max, lat_min, lat_max

    def _find_closest_node(self, pos):
        closest_node = None
        min_distance = float('inf')
        for node, data in self.graph.nodes(data=True):
            node_pos = data['pos']
            dist = np.linalg.norm(pos - node_pos)
            if dist < min_distance:
                closest_node = node
                min_distance = dist
        return closest_node

    def step(self, action_dict):
        rewards = {agent: 0 for agent in self.agents}
        dones = {agent: False for agent in self.agents}
        infos = {}
        truncateds = {agent: False for agent in self.agents}

        for agent, action in action_dict.items():
            if agent in self.done_agents:
                continue

            current_node = self.pos[agent]
            next_node = action

            if current_node in self.graph and next_node in self.graph[current_node]:
                self.pos[agent] = next_node
                self.distance_travelled[agent] += self.graph[current_node][next_node]['weight']

                # Reward for moving towards the destination
                if next_node == self.destinations[agent]:
                    rewards[agent] += 100  # High reward for reaching the destination
                    dones[agent] = True
                    self.done_agents.add(agent)
                else:
                    rewards[agent] += 1  # Reduced positive reward for moving towards the destination
                    rewards[agent] -= 0.5  # Small negative reward for taking a longer path
            else:
                rewards[agent] -= 1  # Penalty for invalid action

        occupied_nodes = {}
        for agent in self.agents:
            if agent in self.done_agents:
                continue

            pos = self.pos[agent]
            if pos not in occupied_nodes:
                occupied_nodes[pos] = agent
            else:
                # Collision occurred
                rewards[agent] -= 50  # High penalty for collision
                other_agent = occupied_nodes[pos]
                rewards[other_agent] -= 50
                dones[agent] = True
                dones[other_agent] = True
                self.done_agents.add(agent)
                self.done_agents.add(other_agent)

        for agent in self.agents:
            if agent not in self.done_agents:
                dones[agent] = False
            else:
                dones[agent] = True
                truncateds[agent] = True

        all_done = len(self.done_agents) == self.total_agents
        dones['__all__'] = all_done
        truncateds['__all__'] = all_done

        obs = {agent: self.pos[agent] for agent in self.agents if agent not in self.done_agents}
        infos = {agent: {} for agent in self.agents if agent not in self.done_agents}

        if self.callback:
            self.callback.update_positions(self.graph, {agent: self.graph.nodes[self.pos[agent]]['pos'] for agent in self.agents})

        # Render the environment state after each step
        self.render()

        return obs, rewards, dones, truncateds, infos

    def render(self):
        # Handle Pygame events to prevent the window from hanging
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        # Draw the graph
        self.screen.fill((0, 0, 0))  # Black background)  # White background

        # Draw edges
        for edge in self.graph.edges():
            start_pos = self.graph.nodes[edge[0]]['pos']
            end_pos = self.graph.nodes[edge[1]]['pos']
            pygame.draw.line(self.screen, (0, 0, 255), start_pos, end_pos, 2)  # Blue for edges  # Light grey for edges

        # Draw nodes
        for node, data in self.graph.nodes(data=True):
            pos = data['pos']
            color = (0, 0, 255) if data.get('is_gate') else (0, 255, 0)
            pygame.draw.circle(self.screen, color, (int(pos[0]), int(pos[1])), 5)

        # Draw agents
        for agent, node_id in self.pos.items():
            if node_id is not None:
                pos = self.graph.nodes[node_id]['pos']
                pygame.draw.circle(self.screen, (255, 0, 0), (int(pos[0]), int(pos[1])), 10)

        pygame.display.flip()
        self.clock.tick(60)  # Cap at 60 frames per second

# ================================
# Main Function
# ================================
if __name__ == "__main__":
    # Parse KML and Build Graph
    file_path = "reduced_nodes.kml"  # Update this path to your KML file
    taxiways, gates = parse_kml(file_path)
    G, node_positions, convert_coords = build_graph(taxiways, gates, WIDTH, HEIGHT)

    # Initialize Ray
    ray.init(ignore_reinit_error=True, log_to_driver=True)

    # Register the Environment
    from ray.tune.registry import register_env

    def env_creator(env_config):
        return RoadIntersectionEnv(env_config)

    register_env("RoadIntersectionEnv", lambda env_config: env_creator(env_config))

    # RLlib Training Configuration
    config = {
        "env": "RoadIntersectionEnv",
        "env_config": {
            "graph": G,
            "num_agents": 3
        },
        "framework": "torch",
        "num_workers": 1,
        "num_gpus": 1,
        "train_batch_size": 4000,
        "sgd_minibatch_size": 128,
        "lr": 1e-3,
        "gamma": 0.99,
        "log_level": "WARN",
    }

    # Train the model
    tune.run(
        "PPO",
        config=config,
        stop={"training_iteration": 50},
        checkpoint_freq=5,
        checkpoint_at_end=True,
        verbose=1,
        storage_path="D:/aniru-windows/Capstone-Implementation/Localrun_ray_checkpoints"  # Save checkpoints to this directory
    )
    checkpoint_path = "D:/aniru-windows/Capstone-Implementation/Checkpoints_3agents_reduced_nodes_3000iter/PPO_RoadIntersectionEnv_d18b0_00000_0_2024-11-19_19-45-24/checkpoint_000029"  # Update with your checkpoint path
    ppo_trainer = PPO(config=config)
    ppo_trainer.restore(checkpoint_path)

    # Create environment for evaluation
    env = RoadIntersectionEnv(config["env_config"])
    obs = env.reset()
    done = {"__all__": False}
    while not done["__all__"]:
        action_dict = {}
        for agent_id, agent_obs in obs.items():
            action = ppo_trainer.compute_single_action(agent_obs, policy_id="default_policy")
            action_dict[agent_id] = action
        obs, rewards, dones, truncateds, infos = env.step(action_dict)
        done = dones

    # Clean up
    ray.shutdown()

(RolloutWorker pid=19040) 2024-12-02 22:34:09,390	WARNING deprecation.py:50 -- DeprecationWarning: `get_agent_ids` has been deprecated. Use `MultiAgentEnv.possible_agents` instead. This will raise an error in the future!
(PPO pid=22100) Trainable.setup took 20.961 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(PPO pid=22100) Install gputil for GPU system monitoring.
(PPO pid=22100) 2024-12-02 22:35:16,923	WARNING rollout_ops.py:115 -- No samples returned from remote workers. If you have a slow environment or model, consider increasing the `sample_timeout_s` or decreasing the `rollout_fragment_length` in `AlgorithmConfig.env_runners().
(PPO pid=22100) 2024-12-02 22:36:26,619	WARNING rollout_ops.py:115 -- No samples returned from remote workers. If you have a slow environment or model, consider increasing the `sample_timeout_s` or decreasing the `rollout_fragment_length` in `AlgorithmConfig.env_runners().
(PPO pid

TypeError: code() argument 13 must be str, not int

In [ ]:
# D:/aniru-windows/Capstone-Implementation/Localrun_ray_checkpoints
#D:/aniru-windows/Capstone-Implementation/Checkpoints_3agents_reduced_nodes_3000iter/PPO_RoadIntersectionEnv_d18b0_00000_0_2024-11-19_19-45-24/checkpoint_000029


'''
    tune.run(
        "PPO",
        config=config,
        stop={"training_iteration": 50},
        checkpoint_freq=5,
        checkpoint_at_end=True,
        verbose=1,
        storage_path="D:/aniru-windows/Capstone-Implementation/Localrun_ray_checkpoints"  # Save checkpoints to this directory
    )

    # Load from a checkpoint and evaluate
    checkpoint_path = "D:/aniru-windows/Capstone-Implementation/Checkpoints_3agents_reduced_nodes_3000iter/PPO_RoadIntersectionEnv_d18b0_00000_0_2024-11-19_19-45-24/checkpoint_000029"  # Update with your checkpoint path
    ppo_trainer = PPO(config=config)
    ppo_trainer.restore(checkpoint_path)

    # Create environment for evaluation
    env = RoadIntersectionEnv(config["env_config"])
    obs = env.reset()
    done = {"__all__": False}
    while not done["__all__"]:
        action_dict = {}
        for agent_id, agent_obs in obs.items():
            action = ppo_trainer.compute_single_action(agent_obs, policy_id="default_policy")
            action_dict[agent_id] = action
        obs, rewards, dones, truncateds, infos = env.step(action_dict)
        done = dones

    # Clean up
    ray.shutdown()
'''

# Some final working codes

In [14]:
import numpy as np
import networkx as nx
import gymnasium as gym
from gymnasium import spaces
import ray
from ray import tune
from ray.rllib.algorithms.ppo import PPOConfig, PPO
from ray.rllib.env.multi_agent_env import MultiAgentEnv
import sys
import os
import xml.etree.ElementTree as ET
import re
import pygame

# ================================
# Define the Distance Function at the Module Level
# ================================
def distance(p1, p2):
    return np.linalg.norm(p1 - p2)

# ================================
# Set Screen Dimensions
# ================================
WIDTH, HEIGHT = 1200, 628

# ================================
# Parse the KML File
# ================================
def parse_kml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    namespace = {'kml': 'http://www.opengis.net/kml/2.2'}

    taxiways = []
    gates = []

    for placemark in root.findall('.//kml:Placemark', namespace):
        # Extract LineStrings (taxiways and runways)
        line_string = placemark.find('.//kml:LineString/kml:coordinates', namespace)
        if line_string is not None:
            line_coords = line_string.text.strip().split()
            path = []
            for coord in line_coords:
                lon, lat, _ = map(float, coord.split(','))
                path.append((lon, lat))
            taxiway = {'path': path}
            name = placemark.find('kml:name', namespace)
            if name is not None:
                placemark_name = name.text.strip()
                taxiway['name'] = placemark_name
            taxiways.append(taxiway)

        # Extract Points (gates)
        point = placemark.find('.//kml:Point/kml:coordinates', namespace)
        if point is not None:
            coord = point.text.strip().split(',')
            lon, lat, _ = map(float, coord)
            gates.append({'position': (lon, lat)})

    return taxiways, gates

# ================================
# Build the NetworkX Graph
# ================================
def build_graph(taxiways, gates, width, height):
    all_taxiway_coords = [coord for taxiway in taxiways for coord in taxiway['path']]
    all_gate_coords = [gate['position'] for gate in gates]
    all_coords = all_taxiway_coords + all_gate_coords

    lon_min = min(coord[0] for coord in all_coords)
    lon_max = max(coord[0] for coord in all_coords)
    lat_min = min(coord[1] for coord in all_coords)
    lat_max = max(coord[1] for coord in all_coords)

    margin = 0.05
    lon_range = lon_max - lon_min
    lat_range = lat_max - lat_min
    lon_min -= lon_range * margin
    lon_max += lon_range * margin
    lat_min -= lat_range * margin
    lat_max += lat_range * margin

    def convert_coords(lon, lat):
        x = (lon - lon_min) / (lon_max - lon_min) * width
        y = height - (lat - lat_min) / (lat_max - lat_min) * height
        return np.array([x, y])

    for taxiway in taxiways:
        pixel_path = [convert_coords(lon, lat) for lon, lat in taxiway['path']]
        taxiway['pixel_path'] = pixel_path

    for gate in gates:
        gate['pixel_position'] = convert_coords(*gate['position'])

    G = nx.Graph()
    node_id_counter = 0
    node_positions = {}
    for idx, taxiway in enumerate(taxiways):
        path = taxiway['pixel_path']
        taxiway_name = taxiway.get('name', '')
        taxiway_nodes = []
        for i, point in enumerate(path):
            node_id = node_id_counter
            node_id_counter += 1
            G.add_node(node_id, pos=point)
            G.nodes[node_id]['taxiway_name'] = taxiway_name
            node_positions[node_id] = point
            taxiway_nodes.append(node_id)
            if i > 0:
                prev_node_id = taxiway_nodes[i - 1]
                G.add_edge(prev_node_id, node_id, weight=distance(point, node_positions[prev_node_id]))
        taxiway['node_ids'] = taxiway_nodes

    # Ensure all edges have a weight of 1
    for edge in G.edges():
        G[edge[0]][edge[1]]['weight'] = 1

    return G, node_positions, convert_coords

# ================================
# Custom Multi-Agent Environment
# ================================
class RoadIntersectionEnv(MultiAgentEnv):
    def __init__(self, config, callback=None):
        super().__init__()
        self.graph = config["graph"]
        self.total_agents = config["num_agents"]
        self.agents = [f"car_{i}" for i in range(self.total_agents)]
        self.pos = {agent: None for agent in self.agents}
        self.distance_travelled = {agent: 0 for agent in self.agents}
        self.destinations = {agent: None for agent in self.agents}
        self.callback = callback

        self.width, self.height = WIDTH, HEIGHT
        self.screen = None

        # Define observation and action spaces
        self.observation_space = spaces.Discrete(len(self.graph.nodes))
        self.action_space = spaces.Discrete(len(self.graph.nodes))

        # Predefined gates and runway endpoints for agents to start and end
        self.gate_coordinates = [
            (77.71046165300096, 13.20355775032989),
            (77.71066072998956, 13.20166801620613),
            (77.70315360361252, 13.20165318995302),
            (77.71305689010458, 13.201637816132),
            (77.70195813939766, 13.20169406641297),
            (77.71149825860202, 13.20165032306983),
            (77.72279066770344, 13.19968338101175),
            (77.7132055646255, 13.20167917723071),
            (77.7089924860226, 13.20357466411958),
            (77.71648635375834, 13.20336974504588)
        ]

        self.runway_endpoints = [
            (77.68686735208298, 13.20718308006773),
            (77.7222951686759, 13.20687066472886),
            (77.69060216489893, 13.18975920793089),
            (77.72642499411846, 13.18944676842937)
        ]

        # Pygame initialization
        pygame.init()
        self.screen = pygame.display.set_mode((self.width, self.height))
        pygame.display.set_caption("Airport Taxiway Management")
        self.clock = pygame.time.Clock()

    def reset(self, seed=None, options=None):
        if seed is not None:
            np.random.seed(seed)

        # Flatten the list of coordinates for use in np.random.choice
        all_coordinates = self.gate_coordinates + self.runway_endpoints

        for agent in self.agents:
            start_coord = all_coordinates[np.random.choice(len(all_coordinates))]
            dest_coord = all_coordinates[np.random.choice(len(all_coordinates))]
            while dest_coord == start_coord:
                dest_coord = all_coordinates[np.random.choice(len(all_coordinates))]

            # Convert start and destination coordinates to node positions
            start_pos = self._convert_coords(*start_coord)
            dest_pos = self._convert_coords(*dest_coord)

            # Find the closest node to the start and destination positions
            start_node = self._find_closest_node(start_pos)
            dest_node = self._find_closest_node(dest_pos)

            self.pos[agent] = start_node
            self.destinations[agent] = dest_node
            self.distance_travelled[agent] = 0

        self.done_agents = set()
        obs = {agent: self.pos[agent] for agent in self.agents}

        if self.callback:
            self.callback.update_positions(self.graph, {agent: self.graph.nodes[self.pos[agent]]['pos'] for agent in self.agents})

        # Render the environment state after reset
        self.render()

        return obs, {}

    def _convert_coords(self, lon, lat):
        lon_min, lon_max, lat_min, lat_max = self._get_bounds()
        x = (lon - lon_min) / (lon_max - lon_min) * self.width
        y = self.height - (lat - lat_min) / (lat_max - lat_min) * self.height
        return np.array([x, y])

    def _get_bounds(self):
        all_coords = self.gate_coordinates + self.runway_endpoints
        lon_min = min(coord[0] for coord in all_coords)
        lon_max = max(coord[0] for coord in all_coords)
        lat_min = min(coord[1] for coord in all_coords)
        lat_max = max(coord[1] for coord in all_coords)
        return lon_min, lon_max, lat_min, lat_max

    def _find_closest_node(self, pos):
        closest_node = None
        min_distance = float('inf')
        for node, data in self.graph.nodes(data=True):
            node_pos = data['pos']
            dist = np.linalg.norm(pos - node_pos)
            if dist < min_distance:
                closest_node = node
                min_distance = dist
        return closest_node

    def step(self, action_dict):
        rewards = {agent: 0 for agent in self.agents}
        dones = {agent: False for agent in self.agents}
        infos = {}
        truncateds = {agent: False for agent in self.agents}

        for agent, action in action_dict.items():
            if agent in self.done_agents:
                continue

            current_node = self.pos[agent]
            next_node = action

            if current_node in self.graph and next_node in self.graph[current_node]:
                self.pos[agent] = next_node
                self.distance_travelled[agent] += self.graph[current_node][next_node]['weight']

                # Reward for moving towards the destination
                if next_node == self.destinations[agent]:
                    rewards[agent] += 100  # High reward for reaching the destination
                    dones[agent] = True
                    self.done_agents.add(agent)
                else:
                    rewards[agent] += 1  # Reduced positive reward for moving towards the destination
                    rewards[agent] -= 0.5  # Small negative reward for taking a longer path
            else:
                rewards[agent] -= 1  # Penalty for invalid action

        occupied_nodes = {}
        for agent in self.agents:
            if agent in self.done_agents:
                continue

            pos = self.pos[agent]
            if pos not in occupied_nodes:
                occupied_nodes[pos] = agent
            else:
                # Collision occurred
                rewards[agent] -= 50  # High penalty for collision
                other_agent = occupied_nodes[pos]
                rewards[other_agent] -= 50
                dones[agent] = True
                dones[other_agent] = True
                self.done_agents.add(agent)
                self.done_agents.add(other_agent)

        for agent in self.agents:
            if agent not in self.done_agents:
                dones[agent] = False
            else:
                dones[agent] = True
                truncateds[agent] = True

        all_done = len(self.done_agents) == self.total_agents
        dones['__all__'] = all_done
        truncateds['__all__'] = all_done

        obs = {agent: self.pos[agent] for agent in self.agents if agent not in self.done_agents}
        infos = {agent: {} for agent in self.agents if agent not in self.done_agents}

        if self.callback:
            self.callback.update_positions(self.graph, {agent: self.graph.nodes[self.pos[agent]]['pos'] for agent in self.agents})

        # Render the environment state after each step
        self.render()

        return obs, rewards, dones, truncateds, infos

    def render(self):
        # Handle Pygame events to prevent the window from hanging
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        # Draw the graph
        self.screen.fill((0, 0, 0))  # Black background

        # Draw edges
        for edge in self.graph.edges():
            start_pos = self.graph.nodes[edge[0]]['pos']
            end_pos = self.graph.nodes[edge[1]]['pos']
            pygame.draw.line(self.screen, (0, 0, 255), start_pos, end_pos, 2)  # Blue for edges

        # Draw agents
        for agent, node_id in self.pos.items():
            if node_id is not None:
                pos = self.graph.nodes[node_id]['pos']
                if agent.startswith('car') and self.pos[agent] in [self._find_closest_node(self._convert_coords(*coord)) for coord in self.gate_coordinates]:
                    pygame.draw.circle(self.screen, (0, 255, 0), (int(pos[0]), int(pos[1])), 5)  # Green for agents originating from gates
            elif agent.startswith('car') and self.pos[agent] in [self._find_closest_node(self._convert_coords(*coord)) for coord in self.runway_endpoints]:
                pygame.draw.circle(self.screen, (255, 165, 0), (int(pos[0]), int(pos[1])), 5)  # Orange for agents originating from runways  # Reduced size for agents  # Red for agents

        pygame.display.flip()
        self.clock.tick(60)  # Cap at 60 frames per second

# ================================
# Main Function
# ================================
if __name__ == "__main__":
    # Parse KML and Build Graph
    file_path = "reduced_nodes.kml"  # Update this path to your KML file
    taxiways, gates = parse_kml(file_path)
    G, node_positions, convert_coords = build_graph(taxiways, gates, WIDTH, HEIGHT)

    # Initialize Ray
    ray.init(ignore_reinit_error=True, log_to_driver=True)

    # Register the Environment
    from ray.tune.registry import register_env

    def env_creator(env_config):
        return RoadIntersectionEnv(env_config)

    register_env("RoadIntersectionEnv", lambda env_config: env_creator(env_config))

    # RLlib Training Configuration
    config = {
        "env": "RoadIntersectionEnv",
        "env_config": {
            "graph": G,
            "num_agents": 20
        },
        "framework": "torch",
        "num_workers": 1,
        "num_gpus": 1,
        "train_batch_size": 4000,
        "sgd_minibatch_size": 128,
        "lr": 1e-3,
        "gamma": 0.99,
        "log_level": "WARN",
    }

    # Train the model
    tune.run(
        "PPO",
        config=config,
        stop={"training_iteration": 50},
        checkpoint_freq=5,
        checkpoint_at_end=True,
        verbose=1,
        storage_path="D:/aniru-windows/Capstone-Implementation/Localrun_ray_checkpoints"  # Save checkpoints to this directory
    )
    
    # Load from a checkpoint and evaluate
    checkpoint_path = "D:/aniru-windows/Capstone-Implementation/Checkpoints_3agents_reduced_nodes_3000iter/PPO_RoadIntersectionEnv_d18b0_00000_0_2024-11-19_19-45-24/checkpoint_000029"  # Update with your checkpoint path
    ppo_trainer = PPO(config=config)
    ppo_trainer.restore(checkpoint_path)
    
    # Create environment for evaluation
    env = RoadIntersectionEnv(config["env_config"])
    obs = env.reset()
    done = {"__all__": False}
    while not done["__all__"]:
        action_dict = {}
        for agent_id, agent_obs in obs.items():
            action = ppo_trainer.compute_single_action(agent_obs, policy_id="default_policy")
            action_dict[agent_id] = action
        obs, rewards, dones, truncateds, infos = env.step(action_dict)
        done = dones
    
    # Clean up
    ray.shutdown()

(RolloutWorker pid=3584) 2024-12-02 22:56:25,067	WARNING deprecation.py:50 -- DeprecationWarning: `get_agent_ids` has been deprecated. Use `MultiAgentEnv.possible_agents` instead. This will raise an error in the future!
(PPO pid=34216) Trainable.setup took 23.817 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(PPO pid=34216) Install gputil for GPU system monitoring.
(PPO pid=34216) 2024-12-02 22:57:34,266	WARNING rollout_ops.py:115 -- No samples returned from remote workers. If you have a slow environment or model, consider increasing the `sample_timeout_s` or decreasing the `rollout_fragment_length` in `AlgorithmConfig.env_runners().
(PPO pid=34216) 2024-12-02 22:59:35,605	WARNING rollout_ops.py:115 -- No samples returned from remote workers. If you have a slow environment or model, consider increasing the `sample_timeout_s` or decreasing the `rollout_fragment_length` in `AlgorithmConfig.env_runners().
(PPO pid=

(raylet) A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: ffffffffffffffff03f6cbc2754bef6216cd55f301000000 Worker ID: d18df3174cf3a852925c458a214a450294887c18c5994577275f69f3 Node ID: 7f24177ba8a519b84d016cf27ce48afa2709d031a9b2af7d79d40190 Worker IP address: 127.0.0.1 Worker port: 61442 Worker PID: 3584 Worker exit type: SYSTEM_ERROR Worker exit detail: Worker exits unexpectedly. Worker exits with an exit code None.


(PPO pid=34216) 2024-12-02 23:03:20,526	ERROR env_runner_group.py:825 -- Failed to stop workers!
(PPO pid=34216) Traceback (most recent call last):
(PPO pid=34216)   File "C:\Users\aniru\miniconda3\envs\env1\Lib\site-packages\ray\rllib\env\env_runner_group.py", line 821, in stop
(PPO pid=34216)     self.foreach_worker(
(PPO pid=34216)   File "C:\Users\aniru\miniconda3\envs\env1\Lib\site-packages\ray\rllib\env\env_runner_group.py", line 909, in foreach_worker
(PPO pid=34216)     FaultTolerantActorManager.handle_remote_call_result_errors(
(PPO pid=34216)   File "C:\Users\aniru\miniconda3\envs\env1\Lib\site-packages\ray\rllib\utils\actor_manager.py", line 638, in handle_remote_call_result_errors
(PPO pid=34216)     raise result_or_error.get()
(PPO pid=34216)   File "C:\Users\aniru\miniconda3\envs\env1\Lib\site-packages\ray\rllib\utils\actor_manager.py", line 792, in _fetch_result
(PPO pid=34216)     result = ray.get(ready)
(PPO pid=34216)              ^^^^^^^^^^^^^^
(PPO pid=34216)   File

KeyboardInterrupt: 

In [2]:
import numpy as np
import networkx as nx
import gymnasium as gym
from gymnasium import spaces
import ray
from ray import tune
from ray.rllib.algorithms.ppo import PPOConfig, PPO
from ray.rllib.env.multi_agent_env import MultiAgentEnv
import sys
import os
import xml.etree.ElementTree as ET
import re
import pygame

os.environ["CUDA_VISIBLE_DEVICES"] = "0" #To be able to call the GPU later in the code


def distance(p1, p2):
    return np.linalg.norm(p1 - p2)

WIDTH, HEIGHT = 1200, 628

'''
-----------------------------------------------------------------------------------------------------------------------------
PARSING THE KML FILE & BUILDING THE GRAPH: 
-----------------------------------------------------------------------------------------------------------------------------
The below function parses a kml file and gets geographical information. This geographical information is then converted to
pixel coordinates. These pixel coordinates are used to then make points of a graph, generated through networkx, the points
are appended to the graph. This graph now stands for the scaled version of the actual airport floor, it is represented by an
adjacency matrix. All edges in the graph have been assigned a weight of '1' for the sake of computational simplicity. 
-----------------------------------------------------------------------------------------------------------------------------
'''
def parse_kml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    namespace = {'kml': 'http://www.opengis.net/kml/2.2'}

    taxiways = []
    gates = []

    for placemark in root.findall('.//kml:Placemark', namespace):
        # Extract LineStrings (taxiways and runways)
        line_string = placemark.find('.//kml:LineString/kml:coordinates', namespace)
        if line_string is not None:
            line_coords = line_string.text.strip().split()
            path = []
            for coord in line_coords:
                lon, lat, _ = map(float, coord.split(','))
                path.append((lon, lat))
            taxiway = {'path': path}
            name = placemark.find('kml:name', namespace)
            if name is not None:
                placemark_name = name.text.strip()
                taxiway['name'] = placemark_name
            taxiways.append(taxiway)

        point = placemark.find('.//kml:Point/kml:coordinates', namespace)
        if point is not None:
            coord = point.text.strip().split(',')
            lon, lat, _ = map(float, coord)
            gates.append({'position': (lon, lat)})

    return taxiways, gates


def build_graph(taxiways, gates, width, height):
    all_taxiway_coords = [coord for taxiway in taxiways for coord in taxiway['path']]
    all_gate_coords = [gate['position'] for gate in gates]
    all_coords = all_taxiway_coords + all_gate_coords

    lon_min = min(coord[0] for coord in all_coords)
    lon_max = max(coord[0] for coord in all_coords)
    lat_min = min(coord[1] for coord in all_coords)
    lat_max = max(coord[1] for coord in all_coords)

    margin = 0.05
    lon_range = lon_max - lon_min
    lat_range = lat_max - lat_min
    lon_min -= lon_range * margin
    lon_max += lon_range * margin
    lat_min -= lat_range * margin
    lat_max += lat_range * margin

    def convert_coords(lon, lat):
        x = (lon - lon_min) / (lon_max - lon_min) * width
        y = height - (lat - lat_min) / (lat_max - lat_min) * height
        return np.array([x, y])

    for taxiway in taxiways:
        pixel_path = [convert_coords(lon, lat) for lon, lat in taxiway['path']]
        taxiway['pixel_path'] = pixel_path

    for gate in gates:
        gate['pixel_position'] = convert_coords(*gate['position'])

    G = nx.Graph()
    node_id_counter = 0
    node_positions = {}
    for idx, taxiway in enumerate(taxiways):
        path = taxiway['pixel_path']
        taxiway_name = taxiway.get('name', '')
        taxiway_nodes = []
        for i, point in enumerate(path):
            node_id = node_id_counter
            node_id_counter += 1
            G.add_node(node_id, pos=point)
            G.nodes[node_id]['taxiway_name'] = taxiway_name
            node_positions[node_id] = point
            taxiway_nodes.append(node_id)
            if i > 0:
                prev_node_id = taxiway_nodes[i - 1]
                G.add_edge(prev_node_id, node_id, weight=distance(point, node_positions[prev_node_id]))
        taxiway['node_ids'] = taxiway_nodes

    for edge in G.edges():
        G[edge[0]][edge[1]]['weight'] = 1

    return G, node_positions, convert_coords


'''
-----------------------------------------------------------------------------------------------------------------------------
THE ENVIRONMENT: 
-----------------------------------------------------------------------------------------------------------------------------
Below is the environment that the agents move around in. The environment stores the agent parameters of position,
observation space and action space. Rewards are being assigned for collisions and arrival at destination as 
expected. However, there is a nominal small negative reward to ensure that planes always resort to taking the shortest path. 
Reset function positions the agents at their assigned starting points and they can run another episode. The rendering is done
through pygame here for real time training visualisation.
-----------------------------------------------------------------------------------------------------------------------------
'''
class RoadIntersectionEnv(MultiAgentEnv):
    def __init__(self, config, callback=None):
        super().__init__()
        self.graph = config["graph"]
        self.total_agents = config["num_agents"]
        self.agents = [f"car_{i}" for i in range(self.total_agents)]
        self.pos = {agent: None for agent in self.agents}
        self.distance_travelled = {agent: 0 for agent in self.agents}
        self.destinations = {agent: None for agent in self.agents}
        self.callback = callback
        self.width, self.height = 1200, 628
        self.screen = None
        self.observation_space = spaces.Discrete(len(self.graph.nodes))
        self.action_space = spaces.Discrete(len(self.graph.nodes))

        self.gate_coordinates = [
            (77.71046165300096, 13.20355775032989),
            (77.71066072998956, 13.20166801620613),
            (77.70315360361252, 13.20165318995302),
            (77.71305689010458, 13.201637816132),
            (77.70195813939766, 13.20169406641297),
            (77.71149825860202, 13.20165032306983),
            (77.72279066770344, 13.19968338101175),
            (77.7132055646255, 13.20167917723071),
            (77.7089924860226, 13.20357466411958),
            (77.71648635375834, 13.20336974504588)
        ]

        self.runway_endpoints = [
            (77.68686735208298, 13.20718308006773),
            (77.7222951686759, 13.20687066472886),
            (77.69060216489893, 13.18975920793089),
            (77.72642499411846, 13.18944676842937)
        ]

        pygame.init()
        self.screen = pygame.display.set_mode((self.width, self.height))
        pygame.display.set_caption("Airport Taxiway Management")
        self.clock = pygame.time.Clock()

    def reset(self, seed=None, options=None):
        if seed is not None:
            np.random.seed(seed)

        all_coordinates = self.gate_coordinates + self.runway_endpoints

        for agent in self.agents:
            start_coord = all_coordinates[np.random.choice(len(all_coordinates))]
            dest_coord = all_coordinates[np.random.choice(len(all_coordinates))]
            while dest_coord == start_coord:
                dest_coord = all_coordinates[np.random.choice(len(all_coordinates))]


            start_pos = self._convert_coords(*start_coord)
            dest_pos = self._convert_coords(*dest_coord)


            start_node = self._find_closest_node(start_pos)
            dest_node = self._find_closest_node(dest_pos)

            self.pos[agent] = start_node
            self.destinations[agent] = dest_node
            self.distance_travelled[agent] = 0

        self.done_agents = set()
        obs = {agent: self.pos[agent] for agent in self.agents}

        if self.callback:
            self.callback.update_positions(self.graph, {agent: self.graph.nodes[self.pos[agent]]['pos'] for agent in self.agents})


        self.render()

        return obs, {}

    def _convert_coords(self, lon, lat):
        lon_min, lon_max, lat_min, lat_max = self._get_bounds()
        x = (lon - lon_min) / (lon_max - lon_min) * self.width
        y = self.height - (lat - lat_min) / (lat_max - lat_min) * self.height
        return np.array([x, y])

    def _get_bounds(self):
        all_coords = self.gate_coordinates + self.runway_endpoints
        lon_min = min(coord[0] for coord in all_coords)
        lon_max = max(coord[0] for coord in all_coords)
        lat_min = min(coord[1] for coord in all_coords)
        lat_max = max(coord[1] for coord in all_coords)
        return lon_min, lon_max, lat_min, lat_max

    def _find_closest_node(self, pos):
        closest_node = None
        min_distance = float('inf')
        for node, data in self.graph.nodes(data=True):
            node_pos = data['pos']
            dist = np.linalg.norm(pos - node_pos)
            if dist < min_distance:
                closest_node = node
                min_distance = dist
        return closest_node

    def step(self, action_dict):
        rewards = {agent: 0 for agent in self.agents}
        dones = {agent: False for agent in self.agents}
        infos = {}
        truncateds = {agent: False for agent in self.agents}

        for agent, action in action_dict.items():
            if agent in self.done_agents:
                continue

            current_node = self.pos[agent]
            next_node = action

            if current_node in self.graph and next_node in self.graph[current_node]:
                self.pos[agent] = next_node
                self.distance_travelled[agent] += self.graph[current_node][next_node]['weight']

n
                if next_node == self.destinations[agent]:
                    rewards[agent] += 100  
                    dones[agent] = True
                    self.done_agents.add(agent)
                else:
                    rewards[agent] += 1  
                    rewards[agent] -= 0.5  
            else:
                rewards[agent] -= 1  

        occupied_nodes = {}
        for agent in self.agents:
            if agent in self.done_agents:
                continue

            pos = self.pos[agent]
            if pos not in occupied_nodes:
                occupied_nodes[pos] = agent
            else:
                rewards[agent] -= 50  
                other_agent = occupied_nodes[pos]
                rewards[other_agent] -= 50
                dones[agent] = True
                dones[other_agent] = True
                self.done_agents.add(agent)
                self.done_agents.add(other_agent)

        for agent in self.agents:
            if agent not in self.done_agents:
                dones[agent] = False
            else:
                dones[agent] = True
                truncateds[agent] = True

        all_done = len(self.done_agents) == self.total_agents
        dones['__all__'] = all_done
        truncateds['__all__'] = all_done

        obs = {agent: self.pos[agent] for agent in self.agents if agent not in self.done_agents}
        infos = {agent: {} for agent in self.agents if agent not in self.done_agents}

        if self.callback:
            self.callback.update_positions(self.graph, {agent: self.graph.nodes[self.pos[agent]]['pos'] for agent in self.agents})

        self.render()

        return obs, rewards, dones, truncateds, infos

    def render(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        self.screen.fill((0, 0, 0)) 

        for edge in self.graph.edges():
            start_pos = self.graph.nodes[edge[0]]['pos']
            end_pos = self.graph.nodes[edge[1]]['pos']
            pygame.draw.line(self.screen, (0, 0, 255), start_pos, end_pos, 2)  # Blue for edges

        for agent, node_id in self.pos.items():
            if node_id is not None:
                pos = self.graph.nodes[node_id]['pos']
                if agent.startswith('car') and self.pos[agent] in [self._find_closest_node(self._convert_coords(*coord)) for coord in self.gate_coordinates]:
                    pygame.draw.circle(self.screen, (0, 255, 0), (int(pos[0]), int(pos[1])), 5)  # Green for agents originating from gates
            elif agent.startswith('car') and self.pos[agent] in [self._find_closest_node(self._convert_coords(*coord)) for coord in self.runway_endpoints]:
                pygame.draw.circle(self.screen, (255, 165, 0), (int(pos[0]), int(pos[1])), 5)  # Orange for agents originating from runways  # Reduced size for agents  # Red for agents

        # for gate in self.gate_coordinates:
        #     gate_pos = self._convert_coords(*gate)
        #     pygame.draw.rect(self.screen, (255, 20, 147), (gate_pos[0] - 3, gate_pos[1] - 3, 6, 6))  # Pink square for gates
        # for runway in self.runway_endpoints:
        #     runway_pos = self._convert_coords(*runway)
        #     pygame.draw.rect(self.screen, (255, 20, 147), (runway_pos[0] - 3, runway_pos[1] - 3, 6, 6))  # Pink square for runways

        pygame.display.flip()
        self.clock.tick(60)  # Cap at 60 frames per second

'''
-----------------------------------------------------------------------------------------------------------------------------
CONFIGURING THE MODEL AND TRAINING: 
-----------------------------------------------------------------------------------------------------------------------------
In the main function below, the learning algorithm chosen is PPO or proximal policy optimization. The trainer is configured 
to train with certain optimizations and parameters under consideration. This section also loads checkpoints from previous
training instances and proceeds training from there, the trainer is half-immune to change in configurations.
-----------------------------------------------------------------------------------------------------------------------------
'''
if __name__ == "__main__":
    # Parse KML and Build Graph
    file_path = "reduced_nodes.kml"  # Update this path to your KML file
    taxiways, gates = parse_kml(file_path)
    G, node_positions, convert_coords = build_graph(taxiways, gates, WIDTH, HEIGHT)

    # Initialize Ray
    ray.init(ignore_reinit_error=True, log_to_driver=True, num_gpus=1)

    # Register the Environment
    from ray.tune.registry import register_env

    def env_creator(env_config):
        return RoadIntersectionEnv(env_config)

    register_env("RoadIntersectionEnv", lambda env_config: env_creator(env_config))

    # RLlib Training Configuration
    config = {
        "env": "RoadIntersectionEnv",
        "env_config": {
            "graph": G,
            "num_agents": 10
        },
        "framework": "torch",
        "num_workers": 1,
        "num_gpus": 1,
        "train_batch_size": 4000,
        "sgd_minibatch_size": 128,
        "lr": 1e-3,
        "gamma": 0.99,
        "log_level": "WARN",
    }

    # Train the model
    tune.run(
        "PPO",
        config=config,
        stop={"training_iteration": 15},
        checkpoint_freq=5,
        checkpoint_at_end=True,
        verbose=1,
        storage_path="D:/aniru-windows/Capstone-Implementation/Localrun_ray_checkpoints"  # Save checkpoints to this directory
    )
    
    # Load from a checkpoint and evaluate
    checkpoint_path = "D:/aniru-windows/Capstone-Implementation/Checkpoints_3agents_reduced_nodes_3000iter/PPO_RoadIntersectionEnv_d18b0_00000_0_2024-11-19_19-45-24/checkpoint_000029"  # Update with your checkpoint path
    ppo_trainer = PPO(config=config)
    ppo_trainer.restore(checkpoint_path)
    
    # Create environment for evaluation
    env = RoadIntersectionEnv(config["env_config"])
    obs = env.reset()
    done = {"__all__": False}
    while not done["__all__"]:
        action_dict = {}
        for agent_id, agent_obs in obs.items():
            action = ppo_trainer.compute_single_action(agent_obs, policy_id="default_policy")
            action_dict[agent_id] = action
        obs, rewards, dones, truncateds, infos = env.step(action_dict)
        done = dones
    
    # Clean up
    ray.shutdown()

(RolloutWorker pid=19192) 2024-12-03 00:09:43,364	WARNING deprecation.py:50 -- DeprecationWarning: `get_agent_ids` has been deprecated. Use `MultiAgentEnv.possible_agents` instead. This will raise an error in the future!
(PPO pid=21000) Trainable.setup took 21.531 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(PPO pid=21000) Install gputil for GPU system monitoring.
(PPO pid=21000) 2024-12-03 00:10:50,967	WARNING rollout_ops.py:115 -- No samples returned from remote workers. If you have a slow environment or model, consider increasing the `sample_timeout_s` or decreasing the `rollout_fragment_length` in `AlgorithmConfig.env_runners().
(PPO pid=21000) 2024-12-03 00:12:52,202	WARNING rollout_ops.py:115 -- No samples returned from remote workers. If you have a slow environment or model, consider increasing the `sample_timeout_s` or decreasing the `rollout_fragment_length` in `AlgorithmConfig.env_runners().
2024-12-

(raylet) A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: ffffffffffffffffc32a4a41b5a80be0c293732b01000000 Worker ID: da4707a34c84651876eeaf6193a7d694d4d02939a1a76647b191cbc8 Node ID: 01ebf7c62a9468fbda2fc231b02af51c88868f3f05d8285fd4ab2d4d Worker IP address: 127.0.0.1 Worker port: 62701 Worker PID: 19192 Worker exit type: SYSTEM_ERROR Worker exit detail: Worker exits unexpectedly. Worker exits with an exit code None.


TuneError: ('Trials did not complete', [PPO_RoadIntersectionEnv_b94b8_00000])

In [ ]:
# D:/aniru-windows/Capstone-Implementation/Localrun_ray_checkpoints
#D:/aniru-windows/Capstone-Implementation/Checkpoints_3agents_reduced_nodes_3000iter/PPO_RoadIntersectionEnv_d18b0_00000_0_2024-11-19_19-45-24/checkpoint_000029


'''
    tune.run(
        "PPO",
        config=config,
        stop={"training_iteration": 50},
        checkpoint_freq=5,
        checkpoint_at_end=True,
        verbose=1,
        storage_path="D:/aniru-windows/Capstone-Implementation/Localrun_ray_checkpoints"  # Save checkpoints to this directory
    )

    # Load from a checkpoint and evaluate
    checkpoint_path = "D:/aniru-windows/Capstone-Implementation/Checkpoints_3agents_reduced_nodes_3000iter/PPO_RoadIntersectionEnv_d18b0_00000_0_2024-11-19_19-45-24/checkpoint_000029"  # Update with your checkpoint path
    ppo_trainer = PPO(config=config)
    ppo_trainer.restore(checkpoint_path)

    # Create environment for evaluation
    env = RoadIntersectionEnv(config["env_config"])
    obs = env.reset()
    done = {"__all__": False}
    while not done["__all__"]:
        action_dict = {}
        for agent_id, agent_obs in obs.items():
            action = ppo_trainer.compute_single_action(agent_obs, policy_id="default_policy")
            action_dict[agent_id] = action
        obs, rewards, dones, truncateds, infos = env.step(action_dict)
        done = dones

    # Clean up
    ray.shutdown()
'''